# Work In Progress (WIP)

* Reduce frames:
  * Remove all frames that have an exposure length of 0 ("exptime_numeric" == 0.0)
  * Remove all frames that don't have an area ('area'.isna()) - This removes all CATALOG frames
  * Reduce frames for all unique datetimes to a single frame, taking the lowest RLEVEL possible
  * Check that it's okay to remove all blocks without a science frame - Assume they are just failed science blocks?
  * Check for Extract Target that all Science Frames for a given block have the same target - therefore I can just take the first one of these?
  * Check observations in the Calibration PropID (and other calibration ones), create a separate file of those patterns / individuals? Add to schedule.
  * Check that just having blocks with science frames in is okay?
<br>

* *__TODO:__* Make a series of observations through the LCO API (space them apart if possible). Download the data from the API, to see if we can link the observation frames through the API fields alone.

* Find the login information for St Andrews and get all the information about our Proposals through the API.

* Look at the timings for incomplete patterns (the ones showing up without any science frames, so I can't automatically work out the targets), to see if they match up to others in the same Request Number?

# Notes

The RLEVEL column is a value identifying how much data reduction has occurred on the frame.
* 0 is raw unprocessed data
* 91 is most recent processed data
* 90 is processed data, with the previous ORAC pipeline (pre 2016)

* CATALOG frames all end in 'e90_cat.fits', and are generated through the SExtractor from the ORAC-DR Pipeline

TODO: Check that every processed frame has an unprocessed 'related_frame'. If this is the case, we should be able to purge all of the processed data.

# Code

## Import Libraries

In [1]:
import json, os, sys, re
import pandas as pd
import datetime as dt
from os.path import join as pathjoin
from numpy import mean, std, floor, log10, finfo
from download_datasets_lco import create_data_name

## Collate Raw Data Files

In [2]:
# Collate all the raw data files in the target directory
def merge_datasets(dir_path):
    dir_path = pathjoin(dir_path)
    datafile_list = filter(lambda x: x.startswith('data'),os.listdir(dir_path))

    df_list = []

    for datafile in datafile_list:
        filepath = pathjoin(dir_path, datafile)
        data = json.load(open(filepath,"r"))
        df_list.append(pd.DataFrame(data).dropna(axis=1, how="all"))
        # Dropna is a fix for some files having empty (all-NaN) columns. Pandas doesn't like this.        
    df = pd.concat(df_list)
    df.reset_index(drop=True, inplace=True)
    return df

dirpath = 'data/lco/coj_2m0a_2016-02-01_2016-08-01'
raw = merge_datasets(dirpath)

## Convert timestamp to datetime object
Straightforward

In [3]:
# Convert the timestamp to a datetime object
def str_to_datetime(date_str):
    try:
        return dt.datetime.strptime(date_str,"%Y-%m-%dT%H:%M:%S.%fZ")
    except:
        return dt.datetime.strptime(date_str,"%Y-%m-%dT%H:%M:%SZ")

raw['datetime'] = raw['DATE_OBS'].apply(str_to_datetime)
raw

,id,basename,area,related_frames,version_set,filename,url,RLEVEL,DAY_OBS,DATE_OBS,...,OBJECT,SITEID,TELID,EXPTIME,FILTER,L1PUBDAT,OBSTYPE,BLKUID,REQNUM,datetime
0,4560853,coj2m002-fs01-20160731-0042-d00,"{'type': 'Polygon', 'coordinates': [[[14.62160...",[19720335],"[{'id': 20838003, 'created': '2020-03-16T13:37...",coj2m002-fs01-20160731-0042-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T06:26:36.154000Z,...,,coj,2m0a,900.000000,air,2016-07-31T06:26:36.154000Z,DARK,88091935.0,NaN,2016-07-31 06:26:36.154
1,19720335,coj2m002-fs01-20160731-0042-d91,"{'type': 'Polygon', 'coordinates': [[[14.62134...","[4560853, 3536542]","[{'id': 20843168, 'created': '2020-03-16T15:23...",coj2m002-fs01-20160731-0042-d91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-07-31,2016-07-31T06:26:36.154000Z,...,,coj,2m0a,900.000000,air,2016-07-31T06:26:36.154000Z,DARK,88091935.0,NaN,2016-07-31 06:26:36.154
2,4560692,coj2m002-fs01-20160731-0041-d00,"{'type': 'Polygon', 'coordinates': [[[10.66148...",[19720342],"[{'id': 20838012, 'created': '2020-03-16T13:37...",coj2m002-fs01-20160731-0041-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T06:10:50.967000Z,...,,coj,2m0a,900.000000,air,2016-07-31T06:10:50.967000Z,DARK,88091935.0,NaN,2016-07-31 06:10:50.967
3,19720342,coj2m002-fs01-20160731-0041-d91,"{'type': 'Polygon', 'coordinates': [[[10.66122...","[4560692, 3536542]","[{'id': 20843175, 'created': '2020-03-16T15:23...",coj2m002-fs01-20160731-0041-d91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-07-31,2016-07-31T06:10:50.967000Z,...,,coj,2m0a,900.000000,air,2016-07-31T06:10:50.967000Z,DARK,88091935.0,NaN,2016-07-31 06:10:50.967
4,4560594,coj2m002-fs01-20160731-0040-d00,"{'type': 'Polygon', 'coordinates': [[[6.701590...",[19720357],"[{'id': 20838028, 'created': '2020-03-16T13:38...",coj2m002-fs01-20160731-0040-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T05:55:05.764000Z,...,,coj,2m0a,900.000000,air,2016-07-31T05:55:05.764000Z,DARK,88091935.0,NaN,2016-07-31 05:55:05.764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20118,146227,coj2m002-fs01-20160201-0003-b00,"{'type': 'Polygon', 'coordinates': [[[179.9397...","[146396, 19714860]","[{'id': 20827059, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0003-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:21:50.351000Z,...,,coj,2m0a,0.000000,air,2016-02-01T05:21:50.351000Z,BIAS,76661666.0,NaN,2016-02-01 05:21:50.351
20119,19714895,coj2m002-fs01-20160201-0002-b91,"{'type': 'Polygon', 'coordinates': [[[178.7851...","[146242, 3536542]","[{'id': 20829619, 'created': '2020-03-16T10:38...",coj2m002-fs01-20160201-0002-b91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-02-01,2016-02-01T05:17:59.153000Z,...,,coj,2m0a,0.000000,air,2016-02-01T05:17:59.153000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:59.153
20120,146242,coj2m002-fs01-20160201-0002-b00,"{'type': 'Polygon', 'coordinates': [[[178.7854...",[19714895],"[{'id': 20827073, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0002-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:17:59.153000Z,...,,coj,2m0a,0.000000,air,2016-02-01T05:17:59.153000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:59.153
20121,146233,coj2m002-fs01-20160201-0001-b00,"{'type': 'Polygon', 'coordinates': [[[178.7854...","[146411, 19714998]","[{'id': 20827126, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0001-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:17:13.377000Z,...,,coj,2m0a,0.000000,air,2016-02-01T05:17:13.377000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:13.377


## Get coordinates of centroid of frame
Take the mean RA and mean DEC of all polygon vertices.
<br>
_NOTE_: Should I be taking halfway between the Min and Max of both coordinates?

In [4]:
# Get the coordinates of the centroid of the frame
def get_centroid(area):
    try:
        coordinates = area.get('coordinates')[0]
    except:
        return pd.Series((None,None))
    if coordinates == None:
        return None
    ra = []
    dec = []
    for corner in coordinates:
        ra.append(corner[0])
        dec.append(corner[1])
    mean_ra = mean(ra)
    if mean_ra < 0:
        mean_ra += 360
    return pd.Series((mean_ra, mean(dec)))

raw[['RA','DEC']] = raw['area'].apply(get_centroid)
raw

,id,basename,area,related_frames,version_set,filename,url,RLEVEL,DAY_OBS,DATE_OBS,...,TELID,EXPTIME,FILTER,L1PUBDAT,OBSTYPE,BLKUID,REQNUM,datetime,RA,DEC
0,4560853,coj2m002-fs01-20160731-0042-d00,"{'type': 'Polygon', 'coordinates': [[[14.62160...",[19720335],"[{'id': 20838003, 'created': '2020-03-16T13:37...",coj2m002-fs01-20160731-0042-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T06:26:36.154000Z,...,2m0a,900.000000,air,2016-07-31T06:26:36.154000Z,DARK,88091935.0,NaN,2016-07-31 06:26:36.154,14.557767,-58.876407
1,19720335,coj2m002-fs01-20160731-0042-d91,"{'type': 'Polygon', 'coordinates': [[[14.62134...","[4560853, 3536542]","[{'id': 20843168, 'created': '2020-03-16T15:23...",coj2m002-fs01-20160731-0042-d91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-07-31,2016-07-31T06:26:36.154000Z,...,2m0a,900.000000,air,2016-07-31T06:26:36.154000Z,DARK,88091935.0,NaN,2016-07-31 06:26:36.154,14.558595,-58.876186
2,4560692,coj2m002-fs01-20160731-0041-d00,"{'type': 'Polygon', 'coordinates': [[[10.66148...",[19720342],"[{'id': 20838012, 'created': '2020-03-16T13:37...",coj2m002-fs01-20160731-0041-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T06:10:50.967000Z,...,2m0a,900.000000,air,2016-07-31T06:10:50.967000Z,DARK,88091935.0,NaN,2016-07-31 06:10:50.967,10.597648,-58.877839
3,19720342,coj2m002-fs01-20160731-0041-d91,"{'type': 'Polygon', 'coordinates': [[[10.66122...","[4560692, 3536542]","[{'id': 20843175, 'created': '2020-03-16T15:23...",coj2m002-fs01-20160731-0041-d91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-07-31,2016-07-31T06:10:50.967000Z,...,2m0a,900.000000,air,2016-07-31T06:10:50.967000Z,DARK,88091935.0,NaN,2016-07-31 06:10:50.967,10.598476,-58.877618
4,4560594,coj2m002-fs01-20160731-0040-d00,"{'type': 'Polygon', 'coordinates': [[[6.701590...",[19720357],"[{'id': 20838028, 'created': '2020-03-16T13:38...",coj2m002-fs01-20160731-0040-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T05:55:05.764000Z,...,2m0a,900.000000,air,2016-07-31T05:55:05.764000Z,DARK,88091935.0,NaN,2016-07-31 05:55:05.764,6.637748,-58.878823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20118,146227,coj2m002-fs01-20160201-0003-b00,"{'type': 'Polygon', 'coordinates': [[[179.9397...","[146396, 19714860]","[{'id': 20827059, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0003-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:21:50.351000Z,...,2m0a,0.000000,air,2016-02-01T05:21:50.351000Z,BIAS,76661666.0,NaN,2016-02-01 05:21:50.351,107.876266,-58.696566
20119,19714895,coj2m002-fs01-20160201-0002-b91,"{'type': 'Polygon', 'coordinates': [[[178.7851...","[146242, 3536542]","[{'id': 20829619, 'created': '2020-03-16T10:38...",coj2m002-fs01-20160201-0002-b91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-02-01,2016-02-01T05:17:59.153000Z,...,2m0a,0.000000,air,2016-02-01T05:17:59.153000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:59.153,178.722735,-58.696325
20120,146242,coj2m002-fs01-20160201-0002-b00,"{'type': 'Polygon', 'coordinates': [[[178.7854...",[19714895],"[{'id': 20827073, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0002-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:17:59.153000Z,...,2m0a,0.000000,air,2016-02-01T05:17:59.153000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:59.153,178.721912,-58.696545
20121,146233,coj2m002-fs01-20160201-0001-b00,"{'type': 'Polygon', 'coordinates': [[[178.7854...","[146411, 19714998]","[{'id': 20827126, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0001-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:17:13.377000Z,...,2m0a,0.000000,air,2016-02-01T05:17:13.377000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:13.377,178.721912,-58.696545


## Convert EXPTIME to numeric

In [5]:
raw["exptime_numeric"] = pd.to_numeric(raw["EXPTIME"])
raw

,id,basename,area,related_frames,version_set,filename,url,RLEVEL,DAY_OBS,DATE_OBS,...,EXPTIME,FILTER,L1PUBDAT,OBSTYPE,BLKUID,REQNUM,datetime,RA,DEC,exptime_numeric
0,4560853,coj2m002-fs01-20160731-0042-d00,"{'type': 'Polygon', 'coordinates': [[[14.62160...",[19720335],"[{'id': 20838003, 'created': '2020-03-16T13:37...",coj2m002-fs01-20160731-0042-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T06:26:36.154000Z,...,900.000000,air,2016-07-31T06:26:36.154000Z,DARK,88091935.0,NaN,2016-07-31 06:26:36.154,14.557767,-58.876407,900.0
1,19720335,coj2m002-fs01-20160731-0042-d91,"{'type': 'Polygon', 'coordinates': [[[14.62134...","[4560853, 3536542]","[{'id': 20843168, 'created': '2020-03-16T15:23...",coj2m002-fs01-20160731-0042-d91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-07-31,2016-07-31T06:26:36.154000Z,...,900.000000,air,2016-07-31T06:26:36.154000Z,DARK,88091935.0,NaN,2016-07-31 06:26:36.154,14.558595,-58.876186,900.0
2,4560692,coj2m002-fs01-20160731-0041-d00,"{'type': 'Polygon', 'coordinates': [[[10.66148...",[19720342],"[{'id': 20838012, 'created': '2020-03-16T13:37...",coj2m002-fs01-20160731-0041-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T06:10:50.967000Z,...,900.000000,air,2016-07-31T06:10:50.967000Z,DARK,88091935.0,NaN,2016-07-31 06:10:50.967,10.597648,-58.877839,900.0
3,19720342,coj2m002-fs01-20160731-0041-d91,"{'type': 'Polygon', 'coordinates': [[[10.66122...","[4560692, 3536542]","[{'id': 20843175, 'created': '2020-03-16T15:23...",coj2m002-fs01-20160731-0041-d91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-07-31,2016-07-31T06:10:50.967000Z,...,900.000000,air,2016-07-31T06:10:50.967000Z,DARK,88091935.0,NaN,2016-07-31 06:10:50.967,10.598476,-58.877618,900.0
4,4560594,coj2m002-fs01-20160731-0040-d00,"{'type': 'Polygon', 'coordinates': [[[6.701590...",[19720357],"[{'id': 20838028, 'created': '2020-03-16T13:38...",coj2m002-fs01-20160731-0040-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T05:55:05.764000Z,...,900.000000,air,2016-07-31T05:55:05.764000Z,DARK,88091935.0,NaN,2016-07-31 05:55:05.764,6.637748,-58.878823,900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20118,146227,coj2m002-fs01-20160201-0003-b00,"{'type': 'Polygon', 'coordinates': [[[179.9397...","[146396, 19714860]","[{'id': 20827059, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0003-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:21:50.351000Z,...,0.000000,air,2016-02-01T05:21:50.351000Z,BIAS,76661666.0,NaN,2016-02-01 05:21:50.351,107.876266,-58.696566,0.0
20119,19714895,coj2m002-fs01-20160201-0002-b91,"{'type': 'Polygon', 'coordinates': [[[178.7851...","[146242, 3536542]","[{'id': 20829619, 'created': '2020-03-16T10:38...",coj2m002-fs01-20160201-0002-b91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-02-01,2016-02-01T05:17:59.153000Z,...,0.000000,air,2016-02-01T05:17:59.153000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:59.153,178.722735,-58.696325,0.0
20120,146242,coj2m002-fs01-20160201-0002-b00,"{'type': 'Polygon', 'coordinates': [[[178.7854...",[19714895],"[{'id': 20827073, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0002-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:17:59.153000Z,...,0.000000,air,2016-02-01T05:17:59.153000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:59.153,178.721912,-58.696545,0.0
20121,146233,coj2m002-fs01-20160201-0001-b00,"{'type': 'Polygon', 'coordinates': [[[178.7854...","[146411, 19714998]","[{'id': 20827126, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0001-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:17:13.377000Z,...,0.000000,air,2016-02-01T05:17:13.377000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:13.377,178.721912,-58.696545,0.0


### Explore

In [6]:
for b, g in raw.groupby("BLKUID"):
    print(g[["RA", "OBSTYPE"]])
    break

               RA OBSTYPE
19870  205.056425    DARK
19886  200.471906    DARK
19906  178.722735    BIAS
19912  107.877132    BIAS
20048  220.823037    DARK
...           ...     ...
20118  107.876266    BIAS
20119  178.722735    BIAS
20120  178.721912    BIAS
20121  178.721912    BIAS
20122  178.722735    BIAS

[79 rows x 2 columns]


## Remove Invalid Frames
Remove frames where the exposure is 0s (not useful), or there is no defined area (CATALOG frames)

In [7]:
def remove_invalid_frames(df):
    df1 = df[df["exptime_numeric"] > 0.0]
    df2 = df1[df1["area"].notna()]
    print("Removed frames with 0s duration:", df.shape, "-->", df1.shape)
    print("Removed frames with no specified area:", df1.shape, "-->", df2.shape)
    return df2
temp1 = remove_invalid_frames(raw)

Removed frames with 0s duration: (20123, 25) --> (15724, 25)
Removed frames with no specified area: (15724, 25) --> (13834, 25)


## Filter out undesired columns
Take only the columns that we're interested in.

In [8]:
# Filter to only the desired columns? What are we throwing away here?
desired_columns = ['datetime','BLKUID','EXPTIME','FILTER','INSTRUME','OBJECT',
    'OBSTYPE','PROPID','REQNUM','RLEVEL','RA','DEC', 'SITEID', 'TELID', "exptime_numeric"]

temp2 = temp1[ desired_columns ]

### Unwanted columns

In [9]:
print("Dropped columns: {}".format([x for x in temp1.columns if x not in desired_columns]))

Dropped columns: ['id', 'basename', 'area', 'related_frames', 'version_set', 'filename', 'url', 'DAY_OBS', 'DATE_OBS', 'L1PUBDAT']


In [10]:
print(temp1.iloc[0])

id                                                           4560853
basename                             coj2m002-fs01-20160731-0042-d00
area               {'type': 'Polygon', 'coordinates': [[[14.62160...
related_frames                                            [19720335]
version_set        [{'id': 20838003, 'created': '2020-03-16T13:37...
filename                     coj2m002-fs01-20160731-0042-d00.fits.fz
url                https://archive-lco-global.s3.amazonaws.com/co...
RLEVEL                                                             0
DAY_OBS                                                   2016-07-31
DATE_OBS                                 2016-07-31T06:26:36.154000Z
PROPID                                                     calibrate
INSTRUME                                                        fs01
OBJECT                                                              
SITEID                                                           coj
TELID                             

## Remove Non-Science Proposal IDs

In [11]:
def is_science_propid(s):
    is_science = bool(re.match('\w+\d{4}\w-\d+', x))
    is_calib = bool(s == "calibrate")
    return is_science or is_calib

def remove_non_science_propids(df, show=False):
    l1 = set(df["PROPID"].unique())
    is_science_propid = lambda x: bool(re.match('\w+\d{4}\w-\d+', x))
    output = df[ df["PROPID"].apply(is_science_propid) ]
    l2 = set(output["PROPID"].unique())

    if show:
        print("Science PropIDs: {}".format(l2), end="\n\n")
        print("Non-Science PropIDs: {}".format(l1 - l2), end="\n\n")
        print("Exposures with Request IDs:")
        for p, g in df[df["PROPID"].isin(l1-l2)].groupby("PROPID"):
            print(f"'{p}'", g["REQNUM"].notna().sum())

    return output

temp3 = remove_non_science_propids(temp2, show=True)

Science PropIDs: {'ANU2015B-003', 'AAS2016A-001', 'LCO2016A-015', 'LCO2015B-018', 'ARI2016A-003', 'KEY2014A-002', 'CON2015B-001', 'LCO2015B-004', 'FTP2016A-001', 'ANU2016A-001', 'CON2016A-003', 'KEY2014A-003', 'LCO2015B-009', 'DDT2015B-010', 'LCO2016A-003', 'FTPEPO2014A-004', 'FTPEPO2014A-003', 'ARI2016A-002', 'TAU2015B-002', 'LCO2015B-012', 'HAW2015A-002', 'LCO2015B-013', 'TAU2016A-001', 'ARI2015A-003', 'LCO2016A-002', 'FTP2015A-001', 'LCOEPO2014B-010', 'ARI2015B-001', 'CLN2016A-003', 'KEY2014A-004'}

Non-Science PropIDs: {'', 'LCOEngineering', 'auto_focus', 'calibrate', 'standard', 'COJ_calib'}

Exposures with Request IDs:
'' 0
'COJ_calib' 410
'LCOEngineering' 0
'auto_focus' 591
'calibrate' 0
'standard' 0


### Exploration

In [12]:
temp2a = temp2[ temp2["PROPID"].isin(['no_proposal', 'standard', 'calibrate', 'LCOEngineering', 'auto_focus', 'COJ_calib']) ]
temp2a["REQNUM"].notna().sum()

1001

In [13]:
temp2["REQNUM"].isna().sum()

4734

In [14]:
for p, g in temp2a.groupby("PROPID"):
    print(p)
    print(g["OBSTYPE"].value_counts())
    print()

COJ_calib
OBSTYPE
SPECTRUM    202
ARC         108
LAMPFLAT    100
Name: count, dtype: int64

LCOEngineering
OBSTYPE
EXPERIMENTAL    286
EXPOSE            1
Name: count, dtype: int64

auto_focus
OBSTYPE
EXPERIMENTAL    825
EXPOSE          205
Name: count, dtype: int64

calibrate
OBSTYPE
SKYFLAT         2085
DARK            1372
EXPERIMENTAL       1
Name: count, dtype: int64

standard
OBSTYPE
STANDARD    329
Name: count, dtype: int64



## Sort frames by Observation Datetime
Can do because it's only one telescope

NOTE: Do we actually need to do this, as we group everything by BLKUID anyway?

In [15]:
# temp5 = temp4.sort_values('datetime').reset_index(drop=True)

## Reduce Frames (ERROR)

ERROR - Reduced to way too few frames. Need to have a further look here.

For any non-calibration PropID, there is usually one 0-RLEVEL frame (There are a couple that are missing a 0-RLEVEL). Because of this, we will take the lowest RLEVEL frame, but we are putting this AFTER the stage where we remove all frames with no EXPTIME or AREA, because the e90_cat files are registering an RLEVEL of 0 even though their name implies they should be 90.


For any non-calibration PropID, there is always one 0-RLEVEL frame. LCO probably just discards the raw data from their calibration frames as they are only useful once they've been processed?

Group the frames by the exact datetime (this should result in one group for each actual exposure taken). Then just take the least processed data frame.
<br>
_NOTE_: Should it be the most? Does it make a difference?

In [16]:
def reduce_frames(df):
    obs_groups = df.groupby('datetime')
    expected_frames = len(obs_groups)

    if expected_frames == len(df):
        print("THIS DOES NOTHING - NO FRAMES REDUCED")

    sole_frames = []
    for d, g in obs_groups:
        minimum_rlevel = g["RLEVEL"].min() # For the odd case where there is no 0-RLEVEL frame.
        raw_frames = g[g["RLEVEL"]==minimum_rlevel]
        
        if len(raw_frames) > 1:
            # There is the odd occurence where there are 2 0-RLEVEL frames (or 90-RLEVEL frames if no 0),
            # but they should have the same attributes for everything except the URL, which we've already dropped.]
            # This just checks that that is definitely the case.
            for c in raw_frames.columns:
                try:
                    if len(raw_frames[c].unique()) > 1:
                        print("ERROR - Duplicate entries with different attributes")
                        print("Column: ", c)
                        print(raw_frames[c])
                        raise Error

                except TypeError:
                    print("Warning - Still trying to compare unhashable data types: ", c)

            sole_frames.append(raw_frames.head(1))

        else:
            sole_frames.append(raw_frames)

    new_df = pd.concat(sole_frames, axis=0)


    resultant_frames = len(new_df)
    if resultant_frames != expected_frames:
        print("Unexpected number of frames returned: {} expected, {} received.".format(\
            expected_frames, resultant_frames))
    return new_df

temp4 = reduce_frames(temp3).reset_index(drop=True)
print(temp3.shape, "-->", temp4.shape)

(8099, 15) --> (4119, 15)


In [17]:
temp4.reset_index(drop=True)

,datetime,BLKUID,EXPTIME,FILTER,INSTRUME,OBJECT,OBSTYPE,PROPID,REQNUM,RLEVEL,RA,DEC,SITEID,TELID,exptime_numeric
0,2016-02-01 10:14:29.486,76701806.0,1800.000000,air,en05,SDSS0153m10,SPECTRUM,KEY2014A-002,487801.0,0,28.323107,-10.003155,coj,2m0a,1800.0
1,2016-02-01 11:14:18.716,76705538.0,30.000000,V,fs01,g16aar,EXPOSE,FTPEPO2014A-004,490067.0,0,44.617453,-46.474192,coj,2m0a,30.0
2,2016-02-01 11:19:01.488,76706489.0,30.000000,V,fs01,NGC1818,EXPOSE,ARI2015B-001,486725.0,0,76.095609,-66.452102,coj,2m0a,30.0
3,2016-02-01 11:20:04.310,76706489.0,30.000000,I,fs01,NGC1818,EXPOSE,ARI2015B-001,486725.0,0,76.095609,-66.452102,coj,2m0a,30.0
4,2016-02-01 11:25:02.636,76707521.0,30.000000,V,fs01,NGC1831,EXPOSE,ARI2015B-001,486743.0,0,76.610291,-64.937112,coj,2m0a,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,2016-07-30 19:26:02.170,88095601.0,80.000000,air,en05,SN 2016egz,ARC,LCO2016A-002,671546.0,0,0.984475,-34.824510,coj,2m0a,80.0
4115,2016-07-30 19:28:09.128,88095601.0,20.000000,air,en05,SN 2016egz,LAMPFLAT,LCO2016A-002,671546.0,0,0.984346,-34.824370,coj,2m0a,20.0
4116,2016-07-30 19:36:00.100,88098658.0,40.000000,V,fs01,ASASSN-16gy,EXPOSE,FTPEPO2014A-004,655782.0,0,35.365868,16.549724,coj,2m0a,40.0
4117,2016-07-30 19:37:09.503,88098658.0,40.000000,R,fs01,ASASSN-16gy,EXPOSE,FTPEPO2014A-004,655782.0,0,35.365868,16.549724,coj,2m0a,40.0


## Extract Science Blocks

### Is Valid Block?

In [18]:
def is_valid_block(block):
    return block["OBSTYPE"].isin(["EXPOSE", "SPECTRUM"]).any()

### Get Largest Intrablock Gap

In [19]:
def get_largest_intrablock_gap(df):
    # Sort rows based on start time
    # Iterate over rows, get gaps
    # Retain largest gap
    sdf = df.sort_values('datetime')
    largest_gap = 0
    for i in range(len(sdf)-1):
        current_ending = sdf.datetime.iloc[i] + dt.timedelta(\
            seconds=float(sdf.EXPTIME.iloc[i]))
        gap = (sdf.datetime.iloc[i+1] - current_ending).total_seconds()
        if gap > largest_gap:
            largest_gap = gap
    return largest_gap

### Extract Target, RA, and DEC (WIP)
* Can we just take the last Target of the block, or only the target of Science Frames (EXPOSE, SPECTRUM)?
* Does it even matter, if we just extract the RA and DEC instead?

In [20]:
def extract_target(block):
    num_targets = len(block.OBJECT.unique())
    
    if num_targets == 1:
        return block.OBJECT.unique()[0]
        
    elif num_targets == 0:
        return None
        
    elif num_targets == 2:
        for index, row in block.iterrows():
            # Take target from first Science frame (Expose or Spectrum)
            # as calibration frames might still be tracking previous targets.
            if row["OBSTYPE"] in ("EXPOSE", "SPECTRUM"):
                return row["OBJECT"]

        # Assuming that we have a block made only of Calibration frames, because the main science frame failed?
        # Need to verify that the pattern matches others in the same Request Number (if possible)
        # Might need manual verification on this one.
        # Look at the timings to pattern match.
        
        
        # Add in exception case for if no science frames are found
        print("ERROR - No valid science target found.")
        print(block)
        return None
                
    else:
        print("ERROR - More than 2 targets found in single block.")
        print(block)
        return None
        
        # if all(x in ('SPECTRUM','ARC','LAMPFLAT') for x in block.OBSTYPE):
        #     if len(block.iloc[2:].OBJECT.unique()) == 1:
        #         # Due to telescope not moving before initial calibration
        #         return block.OBJECT.iloc[-1]
        #     else:
        #         print("FAILED SUBSET CHECK:")
        #         print(block.iloc[2:])
        #         print("")
        #         print(block)
        #         print("\n")
        # else:
        #     print("FAILED OBSTYPE CHECK:")
        #     # print(block[["OBSTYPE", "OBJECT"]].value_counts())
        #     print(block[["OBSTYPE", "OBJECT", "EXPTIME"]])
        #     # print(block["OBSTYPE"].unique())
        #     # print(block["OBJECT"].unique())
        #     # print(block["OBJECT"].value_counts())
        #     # print(block)
        #     print("")

    # If all else fails, return tuple of all targets
    return tuple(sorted(block.OBJECT.unique()))

In [21]:
def extract_target(block):
    science_exposures = ["EXPOSE", "SPECTRUM"]
    science_frames = block[block["OBSTYPE"].isin(science_exposures)]
    num_targets = len(science_frames["OBJECT"].unique())

    if num_targets != 1:
        print("ERROR - Unexpected number of targets: {}".format(num_targets))
        print(block)
        return None

    return block[["OBJECT", "OBSTYPE", "RA", "DEC", "EXPTIME", "datetime"]]

In [68]:
extract_target(block)

,OBJECT,OBSTYPE,RA,DEC,EXPTIME,datetime
2,NGC1818,EXPOSE,76.095609,-66.452102,30.000000,2016-02-01 11:19:01.488
3,NGC1818,EXPOSE,76.095609,-66.452102,30.000000,2016-02-01 11:20:04.310


#### Testing - Compare RA and DEC
The RA and DEC is consistent with the Target (OBJECT) of the frame. As we are assuming that calibration frames may be done on previous targets without moving the telescope, those RAs and DECs should not be included when calculating the mean RA and DEC for the target of the observation.

In [22]:
for blkuid, block in temp4.groupby("BLKUID"):
    num_targets = len(block.OBJECT.unique())
    if num_targets > 1:
        # print(block)
        print(block[["OBJECT", "RA", "DEC"]].iloc[0].to_list())
        break

['AT 2016zb', 145.64080626121648, -16.953072170898103]


#### Testing - Check CATALOG against EXPOSE
Do CATALOG observations always have the same targets as the rest of the block?

RESULT: CATALOG observations never result in multiple targets. They do not appear to be calibration frames.

In [23]:
for blkuid, block in temp4.groupby("BLKUID"):
    exptypes = block["OBSTYPE"].unique()
    num_targets = len(block["OBJECT"].unique())
    if "CATALOG" in exptypes:
        if num_targets > 1:
            print(block)
            print()

#### Testing - Check CATALOG RA and DEC
CATALOG frames do not have valid RA and DEC (from not having a valid 'area' value), so they should all be filtered out.

In [24]:
for i, row in temp4.iterrows():
    if row["OBSTYPE"] == "CATALOG":
        if pd.notna(row["RA"]):
            print(row)
print("Done")

Done


#### Testing - Check CATALOG times relative to other observations

In [25]:
for blkuid, block in temp4.groupby("BLKUID"):
    exptypes = block["OBSTYPE"].unique()
    if ("CATALOG" in exptypes) and ("EXPOSE" not in exptypes):
        print(block[["OBSTYPE", "EXPTIME", "datetime", "OBJECT", "area", "PROPID", "REQNUM"]])
        print()

In [26]:
for blkuid, block in temp4.groupby("REQNUM"):
    exptypes = block["OBSTYPE"].unique()
    if ("CATALOG" in exptypes) and ("EXPOSE" not in exptypes):
        print(block[["OBSTYPE", "EXPTIME", "datetime", "OBJECT", "area", "PROPID", "REQNUM"]])
        print()

### WORK FROM HERE

In [27]:
for blkuid, block in temp4.groupby("BLKUID"):
    extract_target(block)
    # science_exposures = ("EXPOSE", "SPECTRUM", "CATALOG")
    # num_sci_exposures = len(block[block["OBSTYPE"].isin(science_exposures)])
    # if num_sci_exposures < 1:
    #     continue

    # num_sci_targets = len(block[block["OBSTYPE"].isin(science_exposures)]["OBJECT"].unique())
    # num_targets = len(block["OBJECT"].unique())
    
    # if num_targets > 1:
    #     if num_sci_targets != 1:
    #         print(num_targets, num_sci_targets)
    #         print(block["OBSTYPE"].unique())
    
    # # if num_sci_targets > 1:
    #     # print(block)

ERROR - Unexpected number of targets: 0
                  datetime      BLKUID    EXPTIME FILTER INSTRUME   OBJECT  \
28 2016-02-01 14:33:48.960  76714700.0  60.000000    air     en05  1224+04   
29 2016-02-01 14:35:25.986  76714700.0  20.000000    air     en05  1224+04   

     OBSTYPE        PROPID    REQNUM  RLEVEL          RA       DEC SITEID  \
28       ARC  KEY2014A-002  487808.0       0  185.866435  4.970176    coj   
29  LAMPFLAT  KEY2014A-002  487808.0       0  185.866435  4.970176    coj   

   TELID  exptime_numeric  
28  2m0a             60.0  
29  2m0a             20.0  
ERROR - Unexpected number of targets: 0
                  datetime      BLKUID    EXPTIME FILTER INSTRUME   OBJECT  \
32 2016-02-01 15:20:30.710  76716693.0  60.000000    air     en05  1324+03   
33 2016-02-01 15:22:07.167  76716693.0  20.000000    air     en05  1324+03   

     OBSTYPE        PROPID    REQNUM  RLEVEL          RA       DEC SITEID  \
32       ARC  KEY2014A-002  487787.0       0  201.047211 

In [28]:
for blkuid, block in temp4.groupby("BLKUID"):
    if "NGC1818" in block["OBJECT"].unique():
        print(block)
        break

                 datetime      BLKUID    EXPTIME FILTER INSTRUME   OBJECT  \
2 2016-02-01 11:19:01.488  76706489.0  30.000000      V     fs01  NGC1818   
3 2016-02-01 11:20:04.310  76706489.0  30.000000      I     fs01  NGC1818   

  OBSTYPE        PROPID    REQNUM  RLEVEL         RA        DEC SITEID TELID  \
2  EXPOSE  ARI2015B-001  486725.0       0  76.095609 -66.452102    coj  2m0a   
3  EXPOSE  ARI2015B-001  486725.0       0  76.095609 -66.452102    coj  2m0a   

   exptime_numeric  
2             30.0  
3             30.0  


In [29]:
for i, row in temp4.iterrows():
    pass

### Get Pattern (WIP)
* Spectrums often overlap with a lot of their guiding observations
* Catalogs sometimes overlap with some of their exposure observations? Are they different?

In [30]:
def get_pattern(block):
    l1 = len(block)
    sblock = block.sort_values('datetime')
    pattern = []
    for row in sblock[['EXPTIME','INSTRUME','FILTER','OBSTYPE']].itertuples(\
        index=False):
        pattern.append( (float(row[0]),row[1],row[2],row[3]) )
    if len(pattern) < l1:
        print("PROBLEM - mismatched lengths")
    return pattern

### Condense Pattern

In [31]:
def condense_pattern(pattern_tuple):
    condensed_list = []
    current_style = pattern_tuple[0]
    current_count = 0
    for frame in pattern_tuple:
        if current_style == frame:
            current_count += 1
        else:
            condensed_list.append( (current_style,current_count) )
            current_style = frame
            current_count = 1
    condensed_list.append( (current_style, current_count) )
    return tuple(condensed_list)

### Get Pattern Length

In [32]:
def get_pattern_length(pattern_tuple):
    total = 0
    for e in pattern_tuple:
        for _ in range(e[1]):
            total += float(e[0][0])
    return total

### Get Block Times

In [33]:
def get_block_times(block):
    block_times = (block["datetime"] - block["datetime"].iloc[0]).to_list()
    rounded_block_times = [round(i.seconds/30) for i in block_times]
    return rounded_block_times

### Extract all Science Blocks

In [34]:
# Extract contiguous observing blocks
def extract_science_blocks(df):
    # Extract Blocks
    blkuid_groups = df.groupby('BLKUID')
    block_list = []
    
    for blkuid, block in blkuid_groups:

        # Check if the block is a valid science block
        # (It should have at least one EXPOSE or SPECTRUM observation).
        # (If not, assumed it's a failed block).
        if not is_valid_block(block):
            continue
        
        # Get subset of science frames
        block_sci = block[ block['OBSTYPE'].isin(['EXPOSE','SPECTRUM']) ]

        # PropID - Why do we do this and not ReqNum?
        propid_list = [ x for x in block.PROPID.unique() ]
        if len(propid_list) > 1:
            print("ERROR: Block with multiple science propids")
            print(propid_list)
            return None
        propid = propid_list[0]

        # Get the first and last rows of the block
        first_row = block.nsmallest(1,'datetime')
        last_row = block.nlargest(1,'datetime')

        # start_date
        start_date = first_row.datetime.iloc[0]
        
        # duration
        end_date = last_row.datetime.iloc[0] + dt.timedelta(
            seconds=float(last_row.EXPTIME.iloc[0]))
        duration = (end_date - start_date).total_seconds()
        
        # exposure_sum
        exposure_sum = block.EXPTIME.astype(float).sum() + finfo(float).eps
            # NOTE: Exclude blocks of Zero duration
        if exposure_sum == 0.0:
            continue
        
        # science_exposure_sum
        science_exposure_sum = block_sci.EXPTIME.astype(float).sum()
        
        # time_efficiency
        time_efficiency = round(exposure_sum / duration,5)
        
        # exposure_science_efficiency
        exposure_science_efficiency = round(science_exposure_sum / exposure_sum,5)
        
        # total_science_efficiency
        total_science_efficiency = round(science_exposure_sum / duration,5)
        
        # largest_gap
        largest_gap = get_largest_intrablock_gap(block)
        
        # targets
        target = extract_target(block)
        
        # mean_ra and mean_dec - CHANGE THIS
        # TO BE INCLUDED IN EXTRACT TARGET
        mean_ra = block.RA.mean() / 15. # In Hours
        mean_dec = block.DEC.mean()
        
        # moving - Difference of > 4 arcseconds over block
        if (abs(first_row.RA.iloc[0] - last_row.RA.iloc[0]) > 0.001) or \
            (abs(first_row.DEC.iloc[0] - last_row.DEC.iloc[0]) > 0.001):
            moving = True
        else:
            moving = False
       
        # pattern
        pattern = condense_pattern(get_pattern(block))

        # pattern_length
        pattern_length = get_pattern_length(pattern)
        
        # num_exposures
        num_exposures = len(block)
        
        # orphan
        if len(block_sci) == 0:
            orphan = True
        else:
            orphan = False
        
        # reqnum
        reqnum_list = block.REQNUM.unique()
        if len(reqnum_list) > 1:
            reqnum = tuple(sorted(reqnum_list))
        else:
            reqnum = reqnum_list[0]
        
        # instrument
        instrument_list = block.INSTRUME.unique()
        if len(instrument_list) > 1:
            instrument = tuple(sorted(instrument_list))
        else:
            instrument = instrument_list[0]
        
        # science_exposure_times
        science_exposure_times = tuple(block_sci.EXPTIME)

        # SiteIDs
        siteid_list = tuple(block["SITEID"].unique())

        # TelescopeIDs
        telid_list = tuple(block["TELID"].unique())

        # Block Times
        block_times = get_block_times(block)

        block_list.append({
            'blkuid': blkuid,
            'propid': propid,
            'start_date': start_date,
            'duration': duration,
            'exposure_sum': exposure_sum,
            'science_exposure_sum': science_exposure_sum,
            'time_efficiency': time_efficiency,
            'exposure_science_efficiency': exposure_science_efficiency,
            'total_science_efficiency': total_science_efficiency,
            'largest_gap': largest_gap,
            'target': target,
            'mean_ra': mean_ra,
            'mean_dec': mean_dec,
            'moving': moving,
            'pattern': pattern,
            'orphan': orphan,
            'reqnum': reqnum,
            'instrument': instrument,
            'num_exposures': num_exposures,
            'science_exposure_times': science_exposure_times,
            "siteids": siteid_list,
            "telids": telid_list,
            "block_times": block_times,
            "pattern_length": pattern_length,
            "rounded_ra": round(mean_ra, 1),
            "rounded_dec": round(mean_dec, 1)
        })

    return pd.DataFrame(block_list)

block_list = extract_science_blocks(temp4)
block_list

,blkuid,propid,start_date,duration,exposure_sum,science_exposure_sum,time_efficiency,exposure_science_efficiency,total_science_efficiency,largest_gap,...,reqnum,instrument,num_exposures,science_exposure_times,siteids,telids,block_times,pattern_length,rounded_ra,rounded_dec
0,76701806.0,KEY2014A-002,2016-02-01 10:14:29.486,1800.000,1800.0,1800.0,1.00000,1.00000,1.00000,0.000,...,487801.0,en05,1,"(1800.000000,)","(coj,)","(2m0a,)",[0],1800.0,1.9,-10.0
1,76705538.0,FTPEPO2014A-004,2016-02-01 11:14:18.716,30.000,30.0,30.0,1.00000,1.00000,1.00000,0.000,...,490067.0,fs01,1,"(30.000000,)","(coj,)","(2m0a,)",[0],30.0,3.0,-46.5
2,76706489.0,ARI2015B-001,2016-02-01 11:19:01.488,92.822,60.0,60.0,0.64640,1.00000,0.64640,32.822,...,486725.0,fs01,2,"(30.000000, 30.000000)","(coj,)","(2m0a,)","[0, 2]",60.0,5.1,-66.5
3,76707521.0,ARI2015B-001,2016-02-01 11:25:02.636,92.628,60.0,60.0,0.64775,1.00000,0.64775,32.628,...,486743.0,fs01,2,"(30.000000, 30.000000)","(coj,)","(2m0a,)","[0, 2]",60.0,5.1,-64.9
4,76708615.0,FTPEPO2014A-004,2016-02-01 11:32:53.661,419.341,360.0,360.0,0.85849,1.00000,0.85849,29.995,...,489777.0,fs01,3,"(120.000000, 120.000000, 120.000000)","(coj,)","(2m0a,)","[0, 5, 10]",360.0,2.2,-63.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,88088523.0,FTPEPO2014A-004,2016-07-30 16:26:58.961,794.216,720.0,720.0,0.90655,1.00000,0.90655,14.908,...,671312.0,fs01,6,"(120.000000, 120.000000, 120.000000, 120.00000...","(coj,)","(2m0a,)","[0, 4, 9, 13, 18, 22]",720.0,19.5,-25.8
691,88090498.0,KEY2014A-003,2016-07-30 16:49:03.504,1992.352,1900.0,1800.0,0.95365,0.94737,0.90345,50.915,...,671138.0,en05,3,"(1800.000000,)","(coj,)","(2m0a,)","[0, 62, 66]",1900.0,21.9,-34.4
692,88092640.0,KEY2014A-003,2016-07-30 17:30:48.139,2902.053,2800.0,2700.0,0.96483,0.96429,0.93038,55.971,...,671145.0,en05,3,"(2700.000000,)","(coj,)","(2m0a,)","[0, 92, 96]",2800.0,3.1,-69.0
693,88095601.0,LCO2016A-002,2016-07-30 18:25:07.428,3801.700,3700.0,3600.0,0.97325,0.97297,0.94694,54.742,...,671546.0,en05,3,"(3600.000000,)","(coj,)","(2m0a,)","[0, 122, 126]",3700.0,0.1,-34.8


In [35]:
block_list[["pattern", "pattern_length", "total_science_efficiency"]].sort_values("total_science_efficiency")

,pattern,pattern_length,total_science_efficiency
475,"(((3000.0, en05, air, LAMPFLAT), 1), ((80.0, e...",3100.0,0.02527
477,"(((60.0, en05, air, LAMPFLAT), 2), ((341.0, en...",581.0,0.06505
247,"(((60.0, en05, air, LAMPFLAT), 1), ((60.0, en0...",300.0,0.08847
225,"(((60.0, en05, air, LAMPFLAT), 1), ((60.0, en0...",300.0,0.09207
331,"(((60.0, en05, air, LAMPFLAT), 1), ((60.0, en0...",300.0,0.09476
...,...,...,...
254,"(((200.0, fs01, ip, EXPOSE), 1),)",200.0,1.00000
258,"(((120.0, fs01, V, EXPOSE), 1),)",120.0,1.00000
265,"(((240.0, fs01, V, EXPOSE), 1),)",240.0,1.00000
220,"(((240.0, fs01, V, EXPOSE), 1),)",240.0,1.00000


### Testing

#### Exploring Science Blocks

In [36]:
for r, g in block_list.groupby("reqnum"):
    # print(g["siteids"])
    if len(g["siteids"]) > 1:
        # print(g["siteids"])
        print(g["siteids"].unique())
        # break

[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]


In [37]:
g

,blkuid,propid,start_date,duration,exposure_sum,science_exposure_sum,time_efficiency,exposure_science_efficiency,total_science_efficiency,largest_gap,...,reqnum,instrument,num_exposures,science_exposure_times,siteids,telids,block_times,pattern_length,rounded_ra,rounded_dec
693,88095601.0,LCO2016A-002,2016-07-30 18:25:07.428,3801.7,3700.0,3600.0,0.97325,0.97297,0.94694,54.742,...,671546.0,en05,3,"(3600.000000,)","(coj,)","(2m0a,)","[0, 122, 126]",3700.0,0.1,-34.8


In [38]:
print(temp4.columns)
# print(bl1.columns)

Index(['datetime', 'BLKUID', 'EXPTIME', 'FILTER', 'INSTRUME', 'OBJECT',
       'OBSTYPE', 'PROPID', 'REQNUM', 'RLEVEL', 'RA', 'DEC', 'SITEID', 'TELID',
       'exptime_numeric'],
      dtype='object')


#### Ensuring each block only belongs to one request

In [39]:
for b, g in temp4.groupby("BLKUID"):
    if len(g["REQNUM"].unique()) > 1:
        print(b)

#### Checking calibration frames with Expose

In [40]:
for b, g in temp4.groupby("BLKUID"):
    obstypes = g["OBSTYPE"].unique()
    if "EXPOSE" in obstypes:
        print(obstypes)

['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']
['EXPOSE']

#### Checking for multiple science frames in a block

In [41]:
for b, g in temp4.groupby("BLKUID"):
    num_science_frames = g["OBSTYPE"].isin(["EXPOSE", "SPECTRUM"]).sum()
    if num_science_frames > 1:
        print(g[["OBSTYPE", "FILTER", "EXPTIME", "datetime"]])
        print()
        # print(num_science_frames)

  OBSTYPE FILTER    EXPTIME                datetime
2  EXPOSE      V  30.000000 2016-02-01 11:19:01.488
3  EXPOSE      I  30.000000 2016-02-01 11:20:04.310

  OBSTYPE FILTER    EXPTIME                datetime
4  EXPOSE      V  30.000000 2016-02-01 11:25:02.636
5  EXPOSE      I  30.000000 2016-02-01 11:26:05.264

  OBSTYPE FILTER     EXPTIME                datetime
6  EXPOSE      V  120.000000 2016-02-01 11:32:53.661
7  EXPOSE      R  120.000000 2016-02-01 11:35:23.007
8  EXPOSE      I  120.000000 2016-02-01 11:37:53.002

   OBSTYPE FILTER    EXPTIME                datetime
17  EXPOSE      V  30.000000 2016-02-01 13:13:26.820
18  EXPOSE      I  30.000000 2016-02-01 13:14:30.009

   OBSTYPE FILTER    EXPTIME                datetime
19  EXPOSE      V  60.000000 2016-02-01 13:21:48.539
20  EXPOSE      R  60.000000 2016-02-01 13:23:18.288
21  EXPOSE      I  60.000000 2016-02-01 13:24:47.809

   OBSTYPE FILTER    EXPTIME                datetime
30  EXPOSE      V  30.000000 2016-02-01 15:06:4

## Incomplete Pattern Matching (WIP)

In [42]:
for r, g in block_list.groupby("reqnum"):
    # print(g["siteids"])
    if len(g["siteids"]) > 1:
        # print(g["siteids"])
        print(g["siteids"].unique())
        # break

[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]
[('coj',)]


## Full Data Cleaning

In [43]:
def clean_data(data_name):
    data_path = os.path.join("data2", data_name)

    # data_path = os.path.join("data2", "lco", "coj_2m0a_2016-02-01_2016-08-01")
    if not os.path.isdir(data_path):
        print("Could not find relative directory '{}'".format(data_path))
        return
    if not os.path.isfile(pathjoin(data_path,'_complete')):
        print("Data at relative directory '{}' does not have '_complete' file".format(
            data_path))

    # Data exists
    print("Extracting data for files in '{}'...".format(data_name))

    print("Loading Dataframe...")
    raw = merge_datasets(data_path)

    print("- Original Frames: {}".format(len(raw)))
    
    print("Converting dates to datetime objects...")
    raw['datetime'] = raw['DATE_OBS'].apply(str_to_datetime)

    print("Extracting RA and Dec...")
    raw[['RA','DEC']] = raw['area'].apply(get_centroid)

    print("Converting EXPTIME to a numeric value...")
    raw["exptime_numeric"] = pd.to_numeric(raw["EXPTIME"])

    print("Removing frames with no EXPTIME or AREA...")
    df = remove_invalid_frames(raw)
    
    print("Dropping excess columns...")
    df = df[ desired_columns ]

    # print("Filling empty proposal IDs...")
    # df['PROPID'] = df['PROPID'].apply(fill_empty_proposal)

    print("Removing non-science proposals...")
    df = remove_non_science_propids(df)

    print("- Only-Science PropID Frames: {}".format(len(df)))
    
    print("Sorting frames by date...")
    df = df.sort_values('datetime').reset_index(drop=True)

    print("Reducing frames...")
    df = reduce_frames(df)
    print("- Reduced Frames: {}".format(len(df)))
    
    print("Extracting science blocks...")
    block_list = extract_science_blocks(df)

    print("- Science Blocks: {}".format(len(block_list)))

    return (df, block_list, raw)

---

# Extract Data

In [59]:
dirpaths = ['lco/coj_2m0a_2016-02-01_2016-08-01', 'lco/ogg_2m0a_2016-02-01_2016-08-01']

## Extract Data for COJ

In [60]:
df1, bl1, raw1 = clean_data(dirpaths[0])

Extracting data for files in 'lco/coj_2m0a_2016-02-01_2016-08-01'...
Loading Dataframe...
- Original Frames: 20123
Converting dates to datetime objects...
Extracting RA and Dec...
Converting EXPTIME to a numeric value...
Removing frames with no EXPTIME or AREA...
Removed frames with 0s duration: (20123, 25) --> (15724, 25)
Removed frames with no specified area: (15724, 25) --> (13838, 25)
Dropping excess columns...
Removing non-science proposals...
- Only-Science PropID Frames: 8099
Sorting frames by date...
Reducing frames...
- Reduced Frames: 4119
Extracting science blocks...
- Science Blocks: 695


## Extract Data for OGG

In [61]:
df2, bl2, raw2 = clean_data(dirpaths[1])

Extracting data for files in 'lco/ogg_2m0a_2016-02-01_2016-08-01'...
Loading Dataframe...
- Original Frames: 158391
Converting dates to datetime objects...
Extracting RA and Dec...
Converting EXPTIME to a numeric value...
Removing frames with no EXPTIME or AREA...
Removed frames with 0s duration: (158391, 25) --> (148863, 25)
Removed frames with no specified area: (148863, 25) --> (141851, 25)
Dropping excess columns...
Removing non-science proposals...
- Only-Science PropID Frames: 122119
Sorting frames by date...
Reducing frames...
- Reduced Frames: 110486
Extracting science blocks...
- Science Blocks: 1962


In [62]:
print(bl1.shape, bl2.shape, bl1.shape[0]+bl2.shape[0])

(695, 26) (1962, 26) 2657


## Join Dataframes

In [63]:
jbl = pd.concat([bl1, bl2], axis=0)

In [64]:
jbl

,blkuid,propid,start_date,duration,exposure_sum,science_exposure_sum,time_efficiency,exposure_science_efficiency,total_science_efficiency,largest_gap,...,reqnum,instrument,num_exposures,science_exposure_times,siteids,telids,block_times,pattern_length,rounded_ra,rounded_dec
0,76701806.0,KEY2014A-002,2016-02-01 10:14:29.486,1800.000,1800.0,1800.0,1.00000,1.00000,1.00000,0.000,...,487801.0,en05,1,"(1800.000000,)","(coj,)","(2m0a,)",[0],1800.0,1.9,-10.0
1,76705538.0,FTPEPO2014A-004,2016-02-01 11:14:18.716,30.000,30.0,30.0,1.00000,1.00000,1.00000,0.000,...,490067.0,fs01,1,"(30.000000,)","(coj,)","(2m0a,)",[0],30.0,3.0,-46.5
2,76706489.0,ARI2015B-001,2016-02-01 11:19:01.488,92.822,60.0,60.0,0.64640,1.00000,0.64640,32.822,...,486725.0,fs01,2,"(30.000000, 30.000000)","(coj,)","(2m0a,)","[0, 2]",60.0,5.1,-66.5
3,76707521.0,ARI2015B-001,2016-02-01 11:25:02.636,92.628,60.0,60.0,0.64775,1.00000,0.64775,32.628,...,486743.0,fs01,2,"(30.000000, 30.000000)","(coj,)","(2m0a,)","[0, 2]",60.0,5.1,-64.9
4,76708615.0,FTPEPO2014A-004,2016-02-01 11:32:53.661,419.341,360.0,360.0,0.85849,1.00000,0.85849,29.995,...,489777.0,fs01,3,"(120.000000, 120.000000, 120.000000)","(coj,)","(2m0a,)","[0, 5, 10]",360.0,2.2,-63.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,88134994.0,KEY2014A-002,2016-07-31 07:26:44.492,1541.005,2270.0,1200.0,1.47306,0.52863,0.77871,96.322,...,671626.0,"(en06, kb42)",103,"(600.000000, 600.000000)","(ogg,)","(2m0a,)","[0, 0, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6, 6, ...",2270.0,16.2,65.7
1958,88140030.0,KEY2014A-002,2016-07-31 08:16:58.691,2671.759,2520.0,2520.0,0.94320,1.00000,0.94320,30.756,...,672957.0,fs02,8,"(300.000000, 300.000000, 300.000000, 300.00000...","(ogg,)","(2m0a,)","[0, 10, 22, 32, 43, 56, 69, 79]",2520.0,16.2,65.7
1959,88142502.0,FTP2016A-001,2016-07-31 09:09:58.421,200.000,200.0,200.0,1.00000,1.00000,1.00000,0.000,...,536501.0,fs02,1,"(200.000000,)","(ogg,)","(2m0a,)",[0],200.0,17.5,-29.4
1960,88142517.0,KEY2014A-003,2016-07-31 09:20:54.700,3606.364,3400.0,3400.0,0.94278,1.00000,0.94278,35.490,...,672012.0,fs02,10,"(400.000000, 400.000000, 300.000000, 300.00000...","(ogg,)","(2m0a,)","[0, 14, 28, 39, 50, 64, 78, 89, 100, 110]",3400.0,17.3,28.4


In [67]:
for c in jbl.columns:
    print(c)
    print(jbl[c].iloc[0])
    print(jbl[c].dtype)
    print()

blkuid
76701806.0
float64

propid
KEY2014A-002
object

start_date
2016-02-01 10:14:29.486000
datetime64[ns]

duration
1800.0
float64

exposure_sum
1800.0
float64

science_exposure_sum
1800.0
float64

time_efficiency
1.0
float64

exposure_science_efficiency
1.0
float64

total_science_efficiency
1.0
float64

largest_gap
0.0
float64

target
        OBJECT   OBSTYPE         RA        DEC      EXPTIME  \
2  SDSS0153m10  SPECTRUM  28.323107 -10.003155  1800.000000   

                 datetime  
2 2016-02-01 10:14:29.486  
object

mean_ra
1.8882071263581635
float64

mean_dec
-10.00315497413299
float64

moving
False
bool

pattern
(((1800.0, 'en05', 'air', 'SPECTRUM'), 1),)
object

orphan
False
bool

reqnum
487801.0
float64

instrument
en05
object

num_exposures
1
int64

science_exposure_times
('1800.000000',)
object

siteids
('coj',)
object

telids
('2m0a',)
object

block_times
[0]
object

pattern_length
1800.0
float64

rounded_ra
1.9
float64

rounded_dec
-10.0
float64



# Data Exploration

In [ ]:
bl2.sort_values("exposure_science_efficiency", ascending=True).head(10)

In [60]:
bl2["pattern"].value_counts(ascending=False)

pattern
(((200.000000, fs02, ip, EXPOSE), 1),)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

### Check Joint Block List for Request Numbers that span both sites

In [105]:
for r, g in jbl.groupby("reqnum"):
    if len(g["siteids"].unique()) > 1:
        print(g[["pattern", "siteids", "target"]])

                                               pattern siteids   target
51   (((60.000000, en05, air, ARC), 1), ((20.000000...  (coj,)  1324+03
139  (((10.000000, kb42, LL, GUIDE), 24), ((1800.00...  (ogg,)  1324+03
                                               pattern siteids   target
270  (((70.000000, fs01, B, CATALOG), 1), ((70.0000...  (coj,)  NGC2264
225  (((70.000000, fs02, V, EXPOSE), 1), ((70.00000...  (ogg,)  NGC2264
                                               pattern siteids   target
143  (((60.000000, en05, air, ARC), 1), ((20.000000...  (coj,)         
233               (((10.000000, kb42, LL, GUIDE), 7),)  (ogg,)  NGC3227
234  (((10.000000, kb42, LL, GUIDE), 5), ((1200.000...  (ogg,)  NGC3227
                                               pattern siteids   target
173  (((60.000000, en05, air, ARC), 1), ((20.000000...  (coj,)  1324+03
271  (((10.000000, kb42, LL, GUIDE), 4), ((1800.000...  (ogg,)  1324+03
                                               pattern siteids  

In [110]:
for r, g in jbl.groupby("reqnum"):
    s_target = set(g["target"].unique())
    s_target.discard("")
    s_target.discard(None)
    if len(s_target) > 1:
        print(g["target"].unique())
        print(g)
        print()

['L745-46A' 'Feige66']
         blkuid        propid              start_date  duration  exposure_sum  \
322  78490575.0  ANU2015B-003 2016-02-24 15:07:37.112    60.000          60.0   
323  78491163.0  ANU2015B-003 2016-02-24 15:09:43.500   589.237         300.0   

     science_exposure_sum  time_efficiency  exposure_science_efficiency  \
322                   0.0          1.00000                          0.0   
323                  60.0          0.50913                          0.2   

     total_science_efficiency  largest_gap  ...   mean_dec  moving  \
322                   0.00000        0.000  ... -17.411628   False   
323                   0.10183      161.312  ...   8.093719    True   

                                               pattern  orphan    reqnum  \
322           (((60.000000, en05, air, LAMPFLAT), 1),)    True  492924.0   
323  (((60.000000, en05, air, LAMPFLAT), 1), ((60.0...   False  492924.0   

     instrument  num_exposures science_exposure_times  siteids   te

## Checking Requests with Multiple Targets?

In [140]:
test_df = df1

for r, g in test_df.groupby("REQNUM"):
    target_list = set()
    for b, g2 in g.groupby("BLKUID"):
        target_list.add(g2["OBJECT"].unique()[-1])
    target_list.discard("")
    if len(target_list) > 1:
        print(target_list)
        print(g[["datetime", "BLKUID", "OBJECT", "OBSTYPE", "EXPTIME"]])
        print(g[["OBSTYPE", "OBJECT"]].value_counts())

{'Feige66', 'L745-46A'}
                    datetime      BLKUID    OBJECT   OBSTYPE    EXPTIME
1689 2016-02-24 15:07:37.112  78490575.0  L745-46A  LAMPFLAT  60.000000
1690 2016-02-24 15:09:43.500  78491163.0  L745-46A  LAMPFLAT  60.000000
1691 2016-02-24 15:11:45.776  78491163.0  L745-46A       ARC  60.000000
1692 2016-02-24 15:15:27.088  78491163.0   Feige66  SPECTRUM  60.000000
1693 2016-02-24 15:16:56.409  78491163.0   Feige66       ARC  60.000000
1694 2016-02-24 15:18:32.737  78491163.0   Feige66  LAMPFLAT  60.000000
OBSTYPE   OBJECT  
LAMPFLAT  L745-46A    2
ARC       Feige66     1
          L745-46A    1
LAMPFLAT  Feige66     1
SPECTRUM  Feige66     1
Name: count, dtype: int64
{'L745-46A', 'TYC 8907-679-1'}
                    datetime      BLKUID          OBJECT   OBSTYPE     EXPTIME
1655 2016-02-24 11:51:17.202  78479694.0        L745-46A  LAMPFLAT   60.000000
1656 2016-02-24 11:53:18.440  78479694.0        L745-46A       ARC   60.000000
1657 2016-02-24 12:20:11.110  78480665.

In [152]:
valid = []
not_valid = []
for r, g in test_df.groupby("REQNUM"):
    has_valid_block = False
    block_count = 0
    for b, g2 in g.groupby("BLKUID"):
        if g2["OBSTYPE"].isin(["EXPOSE", "SPECTRUM"]).any():
            has_valid_block = True
        block_count += 1

    if block_count > 10:
        print(g)
    
    if has_valid_block:
        valid.append(block_count)
    else:
        not_valid.append(block_count)
        
# print(valid, not_valid)
for i in set(valid):
    print(i, valid.count(i))
print()

for i in set(not_valid):
    print(i, not_valid.count(i))

                    datetime      BLKUID    EXPTIME FILTER INSTRUME   OBJECT  \
3318 2016-05-22 10:13:35.003  83300610.0  60.000000    air     en05  1435+02   
3319 2016-05-22 10:15:20.837  83300610.0  20.000000    air     en05  1435+02   
3320 2016-05-22 11:08:44.097  83303184.0  60.000000    air     en05  1435+02   
3321 2016-05-22 11:10:25.979  83303184.0  20.000000    air     en05  1435+02   
3322 2016-05-22 11:24:22.463  83304895.0  60.000000    air     en05  1435+02   
3323 2016-05-22 11:26:04.272  83304895.0  20.000000    air     en05  1435+02   
3324 2016-05-22 11:35:11.455  83305751.0  60.000000    air     en05  1435+02   
3325 2016-05-22 11:36:53.003  83305751.0  20.000000    air     en05  1435+02   
3326 2016-05-22 11:54:44.848  83306606.0  60.000000    air     en05  1435+02   
3327 2016-05-22 11:56:33.012  83306606.0  20.000000    air     en05  1435+02   
3328 2016-05-22 11:59:53.611  83307037.0  60.000000    air     en05  1435+02   
3329 2016-05-22 12:01:40.079  83307037.0

### Ensuring Blocks only occur at one telescope
At least in this paradigm, where there is little overlap between the telescopes?
It makes no sense for blocks to be spread across multiple telescopes, as the blocks include calibration frames like Arcs and Lampflats, and these are meaningless if not done on the same telescope as the scientific observation.

# Data Products

## RA and Dec Distribution

In [53]:
jbl.columns

Index(['blkuid', 'propid', 'start_date', 'duration', 'exposure_sum',
       'science_exposure_sum', 'time_efficiency',
       'exposure_science_efficiency', 'total_science_efficiency',
       'largest_gap', 'target', 'mean_ra', 'mean_dec', 'moving', 'pattern',
       'orphan', 'reqnum', 'instrument', 'num_exposures',
       'science_exposure_times', 'siteids', 'telids', 'block_times',
       'pattern_length', 'rounded_ra', 'rounded_dec'],
      dtype='object')

In [58]:
jbl["target"]

0               OBJECT   OBSTYPE         RA        DEC...
1          OBJECT OBSTYPE         RA        DEC    EXP...
2            OBJECT OBSTYPE         RA        DEC    E...
3            OBJECT OBSTYPE         RA        DEC    E...
4            OBJECT OBSTYPE         RA        DEC     ...
                              ...                        
1957            OBJECT   OBSTYPE          RA        DE...
1958            OBJECT OBSTYPE          RA        DEC ...
1959                   OBJECT OBSTYPE          RA     ...
1960              OBJECT OBSTYPE          RA        DE...
1961              OBJECT OBSTYPE          RA       DEC...
Name: target, Length: 2657, dtype: object

In [57]:
target_dict = {}
for i, row in jbl.iterrows():
    target = row["target"]
    ra = row["mean_ra"]
    dec = row["mean_dec"]
    print(target)
    print(row)
    break

        OBJECT   OBSTYPE         RA        DEC      EXPTIME  \
2  SDSS0153m10  SPECTRUM  28.323107 -10.003155  1800.000000   

                 datetime  
2 2016-02-01 10:14:29.486  
blkuid                                                                76701806.0
propid                                                              KEY2014A-002
start_date                                            2016-02-01 10:14:29.486000
duration                                                                  1800.0
exposure_sum                                                              1800.0
science_exposure_sum                                                      1800.0
time_efficiency                                                              1.0
exposure_science_efficiency                                                  1.0
total_science_efficiency                                                     1.0
largest_gap                                                                  0.0
target 

In [ ]:
pattern_dict = {}
for p, c in jbl["pattern", "target"].value_counts().to_dict().items():
    for x in p:
        t = x[0][0]
        y = x[0][3]
        r = x[1]
        for _ in range(r):
            print(t, y)
    print()

    translated_pattern = []
    for i in p:
        frame_length = i[0][0]
        instances = i[1]
        # print(frame_length, instances)
        for _ in range(instances):
            translated_pattern.append(frame_length)

    pattern_tuple = tuple(translated_pattern)

    if pattern_tuple not in pattern_dict:
        pattern_dict[pattern_tuple] = {
            "count": 0,
            "targets": []
        }

    pattern_dict[pattern_tuple]["count"] += c
    
sorted(pattern_dict.items())

In [51]:
total = 0

for p, c in pattern_dict.items():
    total += c

patterns = {}
counter = 0
running_total = 0

for p, c in pattern_dict.items():
    print(c)
    running_total += c / total
    patterns[counter] = {
        "pattern": p,
        "count": c,
        "cumulative_probability": running_total
    }
    counter += 1

json.dump(patterns, open("observation_patterns_v2.json", "w"), indent=4)

,blkuid,propid,start_date,duration,exposure_sum,science_exposure_sum,time_efficiency,exposure_science_efficiency,total_science_efficiency,largest_gap,...,reqnum,instrument,num_exposures,science_exposure_times,siteids,telids,block_times,pattern_length,rounded_ra,rounded_dec
0,76701806.0,KEY2014A-002,2016-02-01 10:14:29.486,1800.000,1800.0,1800.0,1.00000,1.00000,1.00000,0.000,...,487801.0,en05,1,"(1800.000000,)","(coj,)","(2m0a,)",[0],1800.0,1.9,-10.0
1,76705538.0,FTPEPO2014A-004,2016-02-01 11:14:18.716,30.000,30.0,30.0,1.00000,1.00000,1.00000,0.000,...,490067.0,fs01,1,"(30.000000,)","(coj,)","(2m0a,)",[0],30.0,3.0,-46.5
2,76706489.0,ARI2015B-001,2016-02-01 11:19:01.488,92.822,60.0,60.0,0.64640,1.00000,0.64640,32.822,...,486725.0,fs01,2,"(30.000000, 30.000000)","(coj,)","(2m0a,)","[0, 2]",60.0,5.1,-66.5
3,76707521.0,ARI2015B-001,2016-02-01 11:25:02.636,92.628,60.0,60.0,0.64775,1.00000,0.64775,32.628,...,486743.0,fs01,2,"(30.000000, 30.000000)","(coj,)","(2m0a,)","[0, 2]",60.0,5.1,-64.9
4,76708615.0,FTPEPO2014A-004,2016-02-01 11:32:53.661,419.341,360.0,360.0,0.85849,1.00000,0.85849,29.995,...,489777.0,fs01,3,"(120.000000, 120.000000, 120.000000)","(coj,)","(2m0a,)","[0, 5, 10]",360.0,2.2,-63.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,88134994.0,KEY2014A-002,2016-07-31 07:26:44.492,1541.005,2270.0,1200.0,1.47306,0.52863,0.77871,96.322,...,671626.0,"(en06, kb42)",103,"(600.000000, 600.000000)","(ogg,)","(2m0a,)","[0, 0, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6, 6, ...",2270.0,16.2,65.7
1958,88140030.0,KEY2014A-002,2016-07-31 08:16:58.691,2671.759,2520.0,2520.0,0.94320,1.00000,0.94320,30.756,...,672957.0,fs02,8,"(300.000000, 300.000000, 300.000000, 300.00000...","(ogg,)","(2m0a,)","[0, 10, 22, 32, 43, 56, 69, 79]",2520.0,16.2,65.7
1959,88142502.0,FTP2016A-001,2016-07-31 09:09:58.421,200.000,200.0,200.0,1.00000,1.00000,1.00000,0.000,...,536501.0,fs02,1,"(200.000000,)","(ogg,)","(2m0a,)",[0],200.0,17.5,-29.4
1960,88142517.0,KEY2014A-003,2016-07-31 09:20:54.700,3606.364,3400.0,3400.0,0.94278,1.00000,0.94278,35.490,...,672012.0,fs02,10,"(400.000000, 400.000000, 300.000000, 300.00000...","(ogg,)","(2m0a,)","[0, 14, 28, 39, 50, 64, 78, 89, 100, 110]",3400.0,17.3,28.4


## Observation Distribution (without Patterns)

In [127]:
jdf = pd.concat([df1, df2], axis=0).reset_index(drop=True)
jdf

,datetime,BLKUID,EXPTIME,FILTER,INSTRUME,OBJECT,OBSTYPE,PROPID,REQNUM,RLEVEL,RA,DEC,SITEID,TELID,exptime_numeric
0,2016-02-01 10:14:29.486,76701806.0,1800.000000,air,en05,SDSS0153m10,SPECTRUM,KEY2014A-002,487801.0,0,28.323107,-10.003155,coj,2m0a,1800.0
1,2016-02-01 11:14:18.716,76705538.0,30.000000,V,fs01,g16aar,CATALOG,FTPEPO2014A-004,490067.0,0,NaN,NaN,coj,2m0a,30.0
2,2016-02-01 11:19:01.488,76706489.0,30.000000,V,fs01,NGC1818,EXPOSE,ARI2015B-001,486725.0,0,76.095609,-66.452102,coj,2m0a,30.0
3,2016-02-01 11:20:04.310,76706489.0,30.000000,I,fs01,NGC1818,CATALOG,ARI2015B-001,486725.0,0,NaN,NaN,coj,2m0a,30.0
4,2016-02-01 11:25:02.636,76707521.0,30.000000,V,fs01,NGC1831,EXPOSE,ARI2015B-001,486743.0,0,76.610291,-64.937112,coj,2m0a,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114652,2016-07-31 12:07:21.065,88146486.0,133.000000,V,fs02,1999 JU3,EXPOSE,LCO2016A-003,671156.0,0,292.205581,-5.173229,ogg,2m0a,133.0
114653,2016-07-31 12:10:04.338,88146486.0,133.000000,R,fs02,1999 JU3,EXPOSE,LCO2016A-003,671156.0,0,292.204543,-5.173259,ogg,2m0a,133.0
114654,2016-07-31 12:12:32.367,88146486.0,133.000000,R,fs02,1999 JU3,EXPOSE,LCO2016A-003,671156.0,0,292.203606,-5.173286,ogg,2m0a,133.0
114655,2016-07-31 12:15:26.020,88146486.0,133.000000,B,fs02,1999 JU3,EXPOSE,LCO2016A-003,671156.0,0,292.202508,-5.173318,ogg,2m0a,133.0


In [130]:
jdf["exptime_numeric"].value_counts().to_dict()

{10.0: 100808,
 60.0: 2651,
 20.0: 2143,
 300.0: 1217,
 200.0: 844,
 6.0: 823,
 120.0: 812,
 80.0: 761,
 30.0: 698,
 600.0: 450,
 133.0: 367,
 180.0: 326,
 100.0: 247,
 400.0: 230,
 1200.0: 217,
 1800.0: 191,
 3600.0: 163,
 15.0: 161,
 35.0: 140,
 55.0: 124,
 40.0: 122,
 50.0: 110,
 150.0: 92,
 2700.0: 91,
 240.0: 91,
 360.0: 84,
 4.0: 84,
 90.0: 66,
 0.0: 52,
 195.0: 48,
 25.0: 42,
 0.8: 40,
 70.0: 35,
 3000.0: 29,
 45.0: 24,
 8.0: 23,
 1500.0: 21,
 5.0: 18,
 125.0: 18,
 250.0: 14,
 233.0: 12,
 140.0: 11,
 29.0: 11,
 1.0: 10,
 900.0: 8,
 143.0: 8,
 78.0: 8,
 121.0: 8,
 46.0: 8,
 210.0: 6,
 82.0: 6,
 28.0: 5,
 32.0: 4,
 41.0: 4,
 232.5: 4,
 3.0: 4,
 130.0: 4,
 95.0: 4,
 700.0: 4,
 260.0: 4,
 220.0: 3,
 105.0: 3,
 12.0: 3,
 500.0: 3,
 2.0: 3,
 480.0: 3,
 170.0: 3,
 160.0: 3,
 321.0: 3,
 4000.0: 3,
 34.0: 2,
 4500.0: 2,
 320.0: 2,
 1100.0: 2,
 9.0: 2,
 1250.0: 1,
 2400.0: 1,
 800.0: 1,
 482.0: 1,
 75.0: 1,
 341.0: 1,
 33.0: 1}

## Pattern Distribution for Request Generation

In [49]:
jbl

,blkuid,propid,start_date,duration,exposure_sum,science_exposure_sum,time_efficiency,exposure_science_efficiency,total_science_efficiency,largest_gap,...,reqnum,instrument,num_exposures,science_exposure_times,siteids,telids,block_times,pattern_length,rounded_ra,rounded_dec
0,76701806.0,KEY2014A-002,2016-02-01 10:14:29.486,1800.000,1800.0,1800.0,1.00000,1.00000,1.00000,0.000,...,487801.0,en05,1,"(1800.000000,)","(coj,)","(2m0a,)",[0],1800.0,1.9,-10.0
1,76705538.0,FTPEPO2014A-004,2016-02-01 11:14:18.716,30.000,30.0,30.0,1.00000,1.00000,1.00000,0.000,...,490067.0,fs01,1,"(30.000000,)","(coj,)","(2m0a,)",[0],30.0,3.0,-46.5
2,76706489.0,ARI2015B-001,2016-02-01 11:19:01.488,92.822,60.0,60.0,0.64640,1.00000,0.64640,32.822,...,486725.0,fs01,2,"(30.000000, 30.000000)","(coj,)","(2m0a,)","[0, 2]",60.0,5.1,-66.5
3,76707521.0,ARI2015B-001,2016-02-01 11:25:02.636,92.628,60.0,60.0,0.64775,1.00000,0.64775,32.628,...,486743.0,fs01,2,"(30.000000, 30.000000)","(coj,)","(2m0a,)","[0, 2]",60.0,5.1,-64.9
4,76708615.0,FTPEPO2014A-004,2016-02-01 11:32:53.661,419.341,360.0,360.0,0.85849,1.00000,0.85849,29.995,...,489777.0,fs01,3,"(120.000000, 120.000000, 120.000000)","(coj,)","(2m0a,)","[0, 5, 10]",360.0,2.2,-63.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,88134994.0,KEY2014A-002,2016-07-31 07:26:44.492,1541.005,2270.0,1200.0,1.47306,0.52863,0.77871,96.322,...,671626.0,"(en06, kb42)",103,"(600.000000, 600.000000)","(ogg,)","(2m0a,)","[0, 0, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6, 6, ...",2270.0,16.2,65.7
1958,88140030.0,KEY2014A-002,2016-07-31 08:16:58.691,2671.759,2520.0,2520.0,0.94320,1.00000,0.94320,30.756,...,672957.0,fs02,8,"(300.000000, 300.000000, 300.000000, 300.00000...","(ogg,)","(2m0a,)","[0, 10, 22, 32, 43, 56, 69, 79]",2520.0,16.2,65.7
1959,88142502.0,FTP2016A-001,2016-07-31 09:09:58.421,200.000,200.0,200.0,1.00000,1.00000,1.00000,0.000,...,536501.0,fs02,1,"(200.000000,)","(ogg,)","(2m0a,)",[0],200.0,17.5,-29.4
1960,88142517.0,KEY2014A-003,2016-07-31 09:20:54.700,3606.364,3400.0,3400.0,0.94278,1.00000,0.94278,35.490,...,672012.0,fs02,10,"(400.000000, 400.000000, 300.000000, 300.00000...","(ogg,)","(2m0a,)","[0, 14, 28, 39, 50, 64, 78, 89, 100, 110]",3400.0,17.3,28.4


In [50]:
jbl["pattern"].value_counts()

pattern
(((200.0, fs02, ip, EXPOSE), 1),)                                                                                                                                                                                                    141
(((10.0, kb42, LL, GUIDE), 5), ((1200.0, en06, air, SPECTRUM), 1), ((10.0, kb42, LL, GUIDE), 92), ((60.0, en06, air, ARC), 1), ((20.0, en06, air, LAMPFLAT), 1))                                                                      65
(((10.0, kb42, LL, GUIDE), 5), ((600.0, en06, air, SPECTRUM), 1), ((10.0, kb42, LL, GUIDE), 47), ((600.0, en06, air, SPECTRUM), 1), ((10.0, kb42, LL, GUIDE), 47), ((60.0, en06, air, ARC), 1), ((20.0, en06, air, LAMPFLAT), 1))     57
(((10.0, kb42, LL, GUIDE), 5), ((600.0, en06, air, SPECTRUM), 1), ((10.0, kb42, LL, GUIDE), 47), ((600.0, en06, air, SPECTRUM), 1), ((10.0, kb42, LL, GUIDE), 46), ((60.0, en06, air, ARC), 1), ((20.0, en06, air, LAMPFLAT), 1))     55
(((300.0, fs02, up, EXPOSE), 2), ((60.0, fs02, gp, EXPOSE), 

In [51]:
for i, row in jbl.iterrows():
    if row["time_efficiency"] > 1:
        exp_types = set()
        for e in row["pattern"]:
            exp_types.add(e[0][3])

        if "CATALOG" in exp_types:
            print(exp_types)
        
            print(row["time_efficiency"])
            print(row["pattern"])
            print(row["block_times"])

            print()

In [52]:
for pattern, group in jbl.groupby("pattern"):
    # print(group.columns)
    # print(pattern)
    
    print(group["time_efficiency"])
    # break

1835    0.04840
1837    0.04798
Name: time_efficiency, dtype: float64
270    0.04717
273    0.04727
276    0.04738
Name: time_efficiency, dtype: float64
73    0.11536
Name: time_efficiency, dtype: float64
1297    0.12946
Name: time_efficiency, dtype: float64
654    0.23081
664    0.23087
Name: time_efficiency, dtype: float64
1933    0.26173
Name: time_efficiency, dtype: float64
1900    0.22789
1935    0.22797
1947    0.22608
1954    0.22806
Name: time_efficiency, dtype: float64
1871    0.21668
Name: time_efficiency, dtype: float64
420     0.20073
568     0.19935
595     0.19710
736     0.19942
1549    0.19590
Name: time_efficiency, dtype: float64
275    0.28877
Name: time_efficiency, dtype: float64
987    0.35618
Name: time_efficiency, dtype: float64
655    0.41613
Name: time_efficiency, dtype: float64
683    0.38686
Name: time_efficiency, dtype: float64
734    1.0
Name: time_efficiency, dtype: float64
737     0.32231
865     0.33118
1088    0.33279
1670    0.33053
Name: time_efficienc

In [53]:
pattern_dict = {}
for p, c in jbl["pattern"].value_counts().to_dict().items():
    for x in p:
        t = x[0][0]
        y = x[0][3]
        r = x[1]
        for _ in range(r):
            print(t, y)
    print()

    translated_pattern = []
    for i in p:
        frame_length = i[0][0]
        instances = i[1]
        # print(frame_length, instances)
        for _ in range(instances):
            translated_pattern.append(frame_length)

    pattern_tuple = tuple(translated_pattern)
    if pattern_tuple not in pattern_dict:
        pattern_dict[pattern_tuple] = 0
    pattern_dict[pattern_tuple] += c
    
sorted(pattern_dict.items())

200.0 EXPOSE

10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
1200.0 SPECTRUM
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10.0 GUIDE
10

[((0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8,
   0.8),
  2),
 ((1.0, 1.0, 1.0), 3),
 ((2.0, 2.0, 5.0, 2.0, 5.0, 5.0), 1),
 ((3.0, 3.0, 3.0), 1),
 ((4.0, 4.0, 4.0, 4.0), 1),
 ((4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0), 6),
 ((4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0,
   4.0),
  1),
 ((5.0, 5.0, 5.0), 5),
 ((6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
   6.0,
 

In [54]:
sorted(pattern_dict.items(), reverse=True, key=lambda x: x[1])

[((200.0,), 274),
 ((200.0, 200.0, 200.0), 114),
 ((10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   1200.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   10.0,
   60.0,
   20.0),
  65),
 ((120.0, 120.0, 120.0), 58),
 ((10.0,
  

In [55]:
total = 0
for p, c in pattern_dict.items():
    total += c
print(total)

2657


In [56]:
import random
rnum = random.randint(0, total)

running_total = 0

for p, c in sorted(pattern_dict.items(), reverse=True, key=lambda x: x[1]):
    running_total += c
    if running_total >= rnum:
        print(p)
        break

(20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.

In [57]:
total = 0

for p, c in pattern_dict.items():
    total += c

patterns = {}
counter = 0
running_total = 0

for p, c in pattern_dict.items():
    print(c)
    running_total += c / total
    patterns[counter] = {
        "pattern": p,
        "count": c,
        "cumulative_probability": running_total
    }
    counter += 1

json.dump(patterns, open("observation_patterns_v2.json", "w"), indent=4)

274
65
57
55
53
114
55
49
48
45
57
37
36
34
34
52
33
33
47
36
28
27
23
22
26
45
40
20
18
27
18
18
17
58
19
15
15
16
19
34
24
12
18
11
19
10
10
19
10
9
9
18
8
8
8
8
8
7
7
10
7
7
6
6
10
5
11
7
5
5
5
5
5
4
7
5
4
4
5
11
5
11
6
4
4
5
4
6
4
4
4
5
5
4
4
5
10
4
3
3
3
4
5
7
3
3
5
3
3
3
3
3
7
3
3
3
3
3
3
3
3
8
2
2
2
2
2
4
2
2
2
2
2
2
2
2
2
6
2
2
2
2
2
2
2
2
2
2
3
2
2
4
4
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
3
2
4
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
3
1
1
1
1
1
1
1
2
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [58]:
t = 0
for i, p in patterns.items():
    print(p["cumulative_probability"])

0.10312382386149793
0.12758750470455402
0.14904027098231087
0.16974030861874295
0.18968761761385022
0.23259315016936397
0.253293187805796
0.2717350395182537
0.28980052691004893
0.30673692133985697
0.32818968761761386
0.3421151674821227
0.35566428302596914
0.3684606699284908
0.38125705683101246
0.40082800150545733
0.4132480240873166
0.42566804666917585
0.4433571697403087
0.45690628528415517
0.4674444862627024
0.47760632292058725
0.48626270229582247
0.4945427173503953
0.5043281896876177
0.5212645841174258
0.5363191569439218
0.5438464433571699
0.5506210011290932
0.560782837786978
0.5675573955589013
0.5743319533308245
0.5807301467820853
0.6025592773805046
0.6097101994730902
0.6153556642830262
0.6210011290929622
0.6270229582235607
0.6341738803161463
0.6469702672186679
0.6560030109145656
0.6605193827625144
0.6672939405344377
0.6714339480617241
0.6785848701543097
0.6823485133609337
0.6861121565675578
0.6932630786601434
0.6970267218667674
0.700414000752729
0.7038012796386905
0.7105758374106138

In [74]:
jbl[["pattern", "mean_ra", "mean_dec", "target"]]

,pattern,mean_ra,mean_dec,target
0,"(((1800.0, en05, air, SPECTRUM), 1),)",1.888207,-10.003155,SDSS0153m10
1,"(((30.0, fs01, V, EXPOSE), 1), ((30.0, fs01, I...",5.073041,-66.452102,NGC1818
2,"(((30.0, fs01, V, EXPOSE), 1), ((30.0, fs01, I...",5.107353,-64.937112,NGC1831
3,"(((120.0, fs01, V, CATALOG), 1), ((120.0, fs01...",2.150202,-63.253123,G16 aar
4,"(((1800.0, en05, air, SPECTRUM), 1), ((80.0, e...",9.709387,-16.953073,AT 2016zb
...,...,...,...,...
1845,"(((10.0, kb42, LL, GUIDE), 5), ((600.0, en06, ...",16.230727,65.710628,Mrk876
1846,"(((300.0, fs02, gp, EXPOSE), 2), ((300.0, fs02...",16.235147,65.702387,Mrk876
1847,"(((200.0, fs02, I, EXPOSE), 1),)",17.478693,-29.379054,XTE J1728-295
1848,"(((400.0, fs02, B, EXPOSE), 2), ((300.0, fs02,...",17.278916,28.353039,ptf16bad


---

# Data Exploration

In [272]:
raw

,id,basename,area,related_frames,version_set,filename,url,RLEVEL,DAY_OBS,DATE_OBS,...,EXPTIME,FILTER,L1PUBDAT,OBSTYPE,BLKUID,REQNUM,datetime,RA,DEC,exptime_numeric
0,4560853,coj2m002-fs01-20160731-0042-d00,"{'type': 'Polygon', 'coordinates': [[[14.62160...",[19720335],"[{'id': 20838003, 'created': '2020-03-16T13:37...",coj2m002-fs01-20160731-0042-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T06:26:36.154000Z,...,900.000000,air,2016-07-31T06:26:36.154000Z,DARK,88091935.0,NaN,2016-07-31 06:26:36.154,14.557767,-58.876407,900.0
1,19720335,coj2m002-fs01-20160731-0042-d91,"{'type': 'Polygon', 'coordinates': [[[14.62134...","[4560853, 3536542]","[{'id': 20843168, 'created': '2020-03-16T15:23...",coj2m002-fs01-20160731-0042-d91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-07-31,2016-07-31T06:26:36.154000Z,...,900.000000,air,2016-07-31T06:26:36.154000Z,DARK,88091935.0,NaN,2016-07-31 06:26:36.154,14.558595,-58.876186,900.0
2,4560692,coj2m002-fs01-20160731-0041-d00,"{'type': 'Polygon', 'coordinates': [[[10.66148...",[19720342],"[{'id': 20838012, 'created': '2020-03-16T13:37...",coj2m002-fs01-20160731-0041-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T06:10:50.967000Z,...,900.000000,air,2016-07-31T06:10:50.967000Z,DARK,88091935.0,NaN,2016-07-31 06:10:50.967,10.597648,-58.877839,900.0
3,19720342,coj2m002-fs01-20160731-0041-d91,"{'type': 'Polygon', 'coordinates': [[[10.66122...","[4560692, 3536542]","[{'id': 20843175, 'created': '2020-03-16T15:23...",coj2m002-fs01-20160731-0041-d91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-07-31,2016-07-31T06:10:50.967000Z,...,900.000000,air,2016-07-31T06:10:50.967000Z,DARK,88091935.0,NaN,2016-07-31 06:10:50.967,10.598476,-58.877618,900.0
4,4560594,coj2m002-fs01-20160731-0040-d00,"{'type': 'Polygon', 'coordinates': [[[6.701590...",[19720357],"[{'id': 20838028, 'created': '2020-03-16T13:38...",coj2m002-fs01-20160731-0040-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T05:55:05.764000Z,...,900.000000,air,2016-07-31T05:55:05.764000Z,DARK,88091935.0,NaN,2016-07-31 05:55:05.764,6.637748,-58.878823,900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,146227,coj2m002-fs01-20160201-0003-b00,"{'type': 'Polygon', 'coordinates': [[[179.9397...","[146396, 19714860]","[{'id': 20827059, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0003-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:21:50.351000Z,...,0.000000,air,2016-02-01T05:21:50.351000Z,BIAS,76661666.0,NaN,2016-02-01 05:21:50.351,107.876266,-58.696566,0.0
119,19714895,coj2m002-fs01-20160201-0002-b91,"{'type': 'Polygon', 'coordinates': [[[178.7851...","[146242, 3536542]","[{'id': 20829619, 'created': '2020-03-16T10:38...",coj2m002-fs01-20160201-0002-b91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-02-01,2016-02-01T05:17:59.153000Z,...,0.000000,air,2016-02-01T05:17:59.153000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:59.153,178.722735,-58.696325,0.0
120,146242,coj2m002-fs01-20160201-0002-b00,"{'type': 'Polygon', 'coordinates': [[[178.7854...",[19714895],"[{'id': 20827073, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0002-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:17:59.153000Z,...,0.000000,air,2016-02-01T05:17:59.153000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:59.153,178.721912,-58.696545,0.0
121,146233,coj2m002-fs01-20160201-0001-b00,"{'type': 'Polygon', 'coordinates': [[[178.7854...","[146411, 19714998]","[{'id': 20827126, 'created': '2020-03-16T09:17...",coj2m002-fs01-20160201-0001-b00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-02-01,2016-02-01T05:17:13.377000Z,...,0.000000,air,2016-02-01T05:17:13.377000Z,BIAS,76661666.0,NaN,2016-02-01 05:17:13.377,178.721912,-58.696545,0.0


In [53]:
raw.columns

Index(['id', 'basename', 'area', 'related_frames', 'version_set', 'filename',
       'url', 'RLEVEL', 'DAY_OBS', 'DATE_OBS', 'PROPID', 'INSTRUME', 'OBJECT',
       'SITEID', 'TELID', 'EXPTIME', 'FILTER', 'L1PUBDAT', 'OBSTYPE', 'BLKUID',
       'REQNUM'],
      dtype='object')

In [54]:
raw["related_frames"]

0              [19720335]
1      [4560853, 3536542]
2              [19720342]
3      [4560692, 3536542]
4              [19720357]
              ...        
118    [146396, 19714860]
119     [146242, 3536542]
120            [19714895]
121    [146411, 19714998]
122     [146233, 3536542]
Name: related_frames, Length: 20123, dtype: object

In [55]:
raw.iloc[0]

id                                                          4560853
basename                            coj2m002-fs01-20160731-0042-d00
area              {'type': 'Polygon', 'coordinates': [[[14.62160...
related_frames                                           [19720335]
version_set       [{'id': 20838003, 'created': '2020-03-16T13:37...
filename                    coj2m002-fs01-20160731-0042-d00.fits.fz
url               https://archive-lco-global.s3.amazonaws.com/co...
RLEVEL                                                            0
DAY_OBS                                                  2016-07-31
DATE_OBS                                2016-07-31T06:26:36.154000Z
PROPID                                                    calibrate
INSTRUME                                                       fs01
OBJECT                                                             
SITEID                                                          coj
TELID                                           

In [57]:
raw[raw["id"].isin([19720335, 4560853, 3536542])]

,id,basename,area,related_frames,version_set,filename,url,RLEVEL,DAY_OBS,DATE_OBS,...,INSTRUME,OBJECT,SITEID,TELID,EXPTIME,FILTER,L1PUBDAT,OBSTYPE,BLKUID,REQNUM
0,4560853,coj2m002-fs01-20160731-0042-d00,"{'type': 'Polygon', 'coordinates': [[[14.62160...",[19720335],"[{'id': 20838003, 'created': '2020-03-16T13:37...",coj2m002-fs01-20160731-0042-d00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-31,2016-07-31T06:26:36.154000Z,...,fs01,,coj,2m0a,900.000000,air,2016-07-31T06:26:36.154000Z,DARK,88091935.0,NaN
1,19720335,coj2m002-fs01-20160731-0042-d91,"{'type': 'Polygon', 'coordinates': [[[14.62134...","[4560853, 3536542]","[{'id': 20843168, 'created': '2020-03-16T15:23...",coj2m002-fs01-20160731-0042-d91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-07-31,2016-07-31T06:26:36.154000Z,...,fs01,,coj,2m0a,900.000000,air,2016-07-31T06:26:36.154000Z,DARK,88091935.0,NaN


In [66]:
for i, r in raw.iterrows():
    print("Prime Frame: {} / {} / {}".format(r["id"], r["OBSTYPE"], r["RLEVEL"]))
    # print(r["id"], r["RLEVEL"])
    
    group = raw[raw["id"].isin(r["related_frames"])]

    for i2, r2 in group.iterrows():
        print("Rel. Frame: {} / {} / {}".format(r2["id"], r2["OBSTYPE"], r2["RLEVEL"]))
    print(r["related_frames"])
    
    # print(group["id"])
    
    # print(i)
    # print(r)
    # print(r["id"])

    
    
    # break
    print()

Prime Frame: 4560853 / DARK / 0
Rel. Frame: 19720335 / DARK / 91
[19720335]

Prime Frame: 19720335 / DARK / 91
Rel. Frame: 4560853 / DARK / 0
[4560853, 3536542]

Prime Frame: 4560692 / DARK / 0
Rel. Frame: 19720342 / DARK / 91
[19720342]

Prime Frame: 19720342 / DARK / 91
Rel. Frame: 4560692 / DARK / 0
[4560692, 3536542]

Prime Frame: 4560594 / DARK / 0
Rel. Frame: 19720357 / DARK / 91
[19720357]

Prime Frame: 19720357 / DARK / 91
Rel. Frame: 4560594 / DARK / 0
[4560594, 3536542]

Prime Frame: 4565944 / DARK / 91
Rel. Frame: 4560357 / DARK / 0
Rel. Frame: 4565943 / BIAS / 91
[4560357, 4565943, 3536542]

Prime Frame: 4560401 / DARK / 0
Rel. Frame: 19720344 / DARK / 91
[19720344]

Prime Frame: 19720344 / DARK / 91
Rel. Frame: 4560401 / DARK / 0
[4560401, 3536542]

Prime Frame: 4560357 / DARK / 0
Rel. Frame: 4565944 / DARK / 91
Rel. Frame: 19720349 / DARK / 91
[4565944, 19720349]

Prime Frame: 19720349 / DARK / 91
Rel. Frame: 4560357 / DARK / 0
[4560357, 3536542]

Prime Frame: 4560285 / D

KeyboardInterrupt: 

In [77]:
raw[raw["id"]==4557664]

,id,basename,area,related_frames,version_set,filename,url,RLEVEL,DAY_OBS,DATE_OBS,...,INSTRUME,OBJECT,SITEID,TELID,EXPTIME,FILTER,L1PUBDAT,OBSTYPE,BLKUID,REQNUM
229,4557664,coj2m002-fs01-20160730-0139-e00,"{'type': 'Polygon', 'coordinates': [[[35.43906...",[4558700],"[{'id': 20837874, 'created': '2020-03-16T13:37...",coj2m002-fs01-20160730-0139-e00.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,0,2016-07-30,2016-07-30T19:38:19.819000Z,...,fs01,ASASSN-16gy,coj,2m0a,40.000000,I,2016-07-30T19:38:19.819000Z,EXPOSE,88098658.0,655782.0


In [78]:
raw[raw["id"]==4558700]

,id,basename,area,related_frames,version_set,filename,url,RLEVEL,DAY_OBS,DATE_OBS,...,INSTRUME,OBJECT,SITEID,TELID,EXPTIME,FILTER,L1PUBDAT,OBSTYPE,BLKUID,REQNUM
228,4558700,coj2m002-fs01-20160730-0139-e91,"{'type': 'Polygon', 'coordinates': [[[35.43389...","[4557664, 4558630, 4558631, 4532523, 3536541]","[{'id': 4769041, 'created': '2016-07-31T00:05:...",coj2m002-fs01-20160730-0139-e91.fits.fz,https://archive-lco-global.s3.amazonaws.com/co...,91,2016-07-30,2016-07-30T19:38:19.819000Z,...,fs01,ASASSN-16gy,coj,2m0a,40.000000,I,2016-07-30T19:38:19.819000Z,EXPOSE,88098658.0,655782.0


In [ ]:
def reduce_frames(df):
    obs_groups = df.groupby('datetime')
    expected_frames = len(obs_groups)

    new_df = pd.concat([
        obs_frames[ obs_frames['RLEVEL'] == obs_frames['RLEVEL'].max() ].head(1)
        for d, obs_frames in obs_groups ]
    )

    resultant_frames = len(new_df)
    if resultant_frames != expected_frames:
        print("Unexpected number of frames returned: {} expected, {} received.".format(\
            expected_frames, resultant_frames))
    return new_df

df = reduce_frames(df)

In [92]:
obs_groups = raw.groupby('datetime')
expected_frames = len(obs_groups)
print(expected_frames)
new_df = pd.concat([
    obs_frames[ obs_frames['RLEVEL'] == obs_frames['RLEVEL'].min() ].head(1)
    for d, obs_frames in obs_groups ]
)

10627


In [93]:
new_df["RLEVEL"].unique()

array([ 0, 91], dtype=int64)

# Comparing with current Archive API

In [1]:
import requests
import json
import urllib
import pandas as pd
import numpy as np

def test_archive_api():
    base_url = "https://archive-api.lco.global/frames/?"
    param_dict = {
        "start": "2016-02-01 00:00",
        "end": "2016-08-01 00:00",
        "public": "true",
        "SITEID": "coj",
        "TELID": "1m0a",
        "limit": 100,
        "offset": 0
    }
    request_url = base_url + urllib.parse.urlencode(param_dict)
    r = requests.get(url=request_url)
    df = pd.DataFrame(r.json()["results"])
    return df

new_df = test_archive_api()

In [2]:
new_df

,id,basename,observation_date,observation_day,proposal_id,instrument_id,target_name,reduction_level,site_id,telescope_id,...,SITEID,TELID,EXPTIME,FILTER,L1PUBDAT,OBSTYPE,BLKUID,REQNUM,area,related_frames
0,4565855,coj1m003-kb77-20160731-0300-d00,2016-07-31T22:23:39.951000Z,2016-07-31,calibrate,kb77,,0,coj,1m0a,...,coj,1m0a,299.236,R,2016-07-31T22:23:39.951000Z,DARK,88091929,NaN,"{'type': 'Polygon', 'coordinates': [[[58.67261...",[]
1,4565846,coj1m003-kb77-20160731-0299-d00,2016-07-31T22:18:24.589000Z,2016-07-31,calibrate,kb77,,0,coj,1m0a,...,coj,1m0a,299.240,R,2016-07-31T22:18:24.589000Z,DARK,88091929,NaN,"{'type': 'Polygon', 'coordinates': [[[57.35386...",[]
2,4565776,coj1m003-kb77-20160731-0298-d00,2016-07-31T22:13:09.198000Z,2016-07-31,calibrate,kb77,,0,coj,1m0a,...,coj,1m0a,299.236,R,2016-07-31T22:13:09.198000Z,DARK,88091929,NaN,"{'type': 'Polygon', 'coordinates': [[[56.03526...",[]
3,4565777,coj1m003-kb77-20160731-0297-d00,2016-07-31T22:07:53.758000Z,2016-07-31,calibrate,kb77,,0,coj,1m0a,...,coj,1m0a,299.237,R,2016-07-31T22:07:53.758000Z,DARK,88091929,NaN,"{'type': 'Polygon', 'coordinates': [[[54.71614...",[]
4,4565697,coj1m003-kb77-20160731-0296-d00,2016-07-31T22:02:38.295000Z,2016-07-31,calibrate,kb77,,0,coj,1m0a,...,coj,1m0a,299.236,R,2016-07-31T22:02:38.295000Z,DARK,88091929,NaN,"{'type': 'Polygon', 'coordinates': [[[53.39417...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4564796,coj1m003-kb77-20160731-0237-e00,2016-07-31T18:42:17.761000Z,2016-07-31,CON2016A-005,kb77,SDSS234540.44-093610.1,0,coj,1m0a,...,coj,1m0a,99.695,rp,2017-07-31T18:42:17.761000Z,EXPOSE,88174203,614350.0,"{'type': 'Polygon', 'coordinates': [[[-3.43029...",[4567330]
96,4567330,coj1m003-kb77-20160731-0237-e91,2016-07-31T18:42:17.761000Z,2016-07-31,CON2016A-005,kb77,SDSS234540.44-093610.1,91,coj,1m0a,...,coj,1m0a,99.695,rp,2017-07-31T18:42:17.761000Z,EXPOSE,88174203,614350.0,"{'type': 'Polygon', 'coordinates': [[[-3.42959...","[4564796, 4566946, 4566945, 4532857, 4380129]"
97,4564790,coj1m003-kb77-20160731-0236-e00,2016-07-31T18:40:29.161000Z,2016-07-31,CON2016A-005,kb77,SDSS234540.44-093610.1,0,coj,1m0a,...,coj,1m0a,99.693,rp,2017-07-31T18:40:29.161000Z,EXPOSE,88174203,614350.0,"{'type': 'Polygon', 'coordinates': [[[-3.43029...",[4567329]
98,4567329,coj1m003-kb77-20160731-0236-e91,2016-07-31T18:40:29.161000Z,2016-07-31,CON2016A-005,kb77,SDSS234540.44-093610.1,91,coj,1m0a,...,coj,1m0a,99.693,rp,2017-07-31T18:40:29.161000Z,EXPOSE,88174203,614350.0,"{'type': 'Polygon', 'coordinates': [[[-3.42958...","[4564790, 4566946, 4566945, 4532857, 4380129]"


In [9]:
new_df.columns

Index(['id', 'basename', 'observation_date', 'observation_day', 'proposal_id',
       'instrument_id', 'target_name', 'reduction_level', 'site_id',
       'telescope_id', 'exposure_time', 'primary_optical_element',
       'public_date', 'configuration_type', 'observation_id', 'request_id',
       'version_set', 'url', 'filename', 'DATE_OBS', 'DAY_OBS', 'PROPID',
       'INSTRUME', 'OBJECT', 'RLEVEL', 'SITEID', 'TELID', 'EXPTIME', 'FILTER',
       'L1PUBDAT', 'OBSTYPE', 'BLKUID', 'REQNUM', 'area', 'related_frames'],
      dtype='object')

In [10]:
raw.columns

Index(['id', 'basename', 'area', 'related_frames', 'version_set', 'filename',
       'url', 'RLEVEL', 'DAY_OBS', 'DATE_OBS', 'PROPID', 'INSTRUME', 'OBJECT',
       'SITEID', 'TELID', 'EXPTIME', 'FILTER', 'L1PUBDAT', 'OBSTYPE', 'BLKUID',
       'REQNUM', 'datetime', 'RA', 'DEC', 'exptime_numeric'],
      dtype='object')

In [11]:
print([x for x in new_df.columns if x not in raw.columns])
print()
print([x for x in raw.columns if x not in new_df.columns])

['observation_date', 'observation_day', 'proposal_id', 'instrument_id', 'target_name', 'reduction_level', 'site_id', 'telescope_id', 'exposure_time', 'primary_optical_element', 'public_date', 'configuration_type', 'observation_id', 'request_id']

['datetime', 'RA', 'DEC', 'exptime_numeric']


In [13]:
new_df.iloc[0]

id                                                                   4565855
basename                                     coj1m003-kb77-20160731-0300-d00
observation_date                                 2016-07-31T22:23:39.951000Z
observation_day                                                   2016-07-31
proposal_id                                                        calibrate
instrument_id                                                           kb77
target_name                                                                 
reduction_level                                                            0
site_id                                                                  coj
telescope_id                                                            1m0a
exposure_time                                                        299.236
primary_optical_element                                                    R
public_date                                      2016-07-31T22:23:39.951000Z